## STEP 0 – Environment & Project Bootstrap
Creates project folder, baseline files, installs minimal Python packages, checks ffmpeg/ffprobe.

In [2]:
# Step 0 — Final Bootstrap (explicit installs + env + plan sync + optional Pillow test)
# Safe to rerun. Does NOT overwrite existing API values in .env.
#
# Features:
#   - Installs pyyaml, pillow, ipywidgets, python-dotenv (unless SKIP_INSTALL=True)
#   - Loads .env
#   - Creates project folder layout
#   - Seeds text.txt if missing
#   - Maintains plan_initial.yaml (only created if missing)
#   - Synchronizes plan_current.yaml scenes to text.txt
#   - Verifies ffmpeg / ffprobe availability
#   - Prints API key status and asset counts
#   - Optional Pillow sanity test (THUMB_TEST=True)
#
# Change PROJECT_DIR if needed; or set env MY_VIDEO_PROJECT_DIR before running.
# After this: run Step 1 editor (auto-assign + preview).

import os, sys, json, shutil, datetime, subprocess
from pathlib import Path

# ---------------- User Config ----------------
main_dir = r"C:/Users/flyre/mynotebooks"
PROJECT_DIR = Path(os.environ.get("MY_VIDEO_PROJECT_DIR", r"C:\Users\flyre\mynotebooks")).expanduser()
SKIP_INSTALL = False          # Set True to skip pip installs (if environment already prepared)
THUMB_TEST   = True           # Create & load a small test image via Pillow to confirm it's working
REQUIRED_PACKAGES = ["pyyaml", "pillow", "ipywidgets", "python-dotenv"]

# ---------------- Paths ----------------
ASSETS_DIR  = PROJECT_DIR / "assets"
FONTS_DIR   = PROJECT_DIR / "fonts"
ENV_FILE    = PROJECT_DIR / ".env"
TEXT_FILE   = PROJECT_DIR / "text.txt"
PLAN_INIT_Y = PROJECT_DIR / "plan_initial.yaml"
PLAN_INIT_J = PROJECT_DIR / "plan_initial.json"
PLAN_CUR_Y  = PROJECT_DIR / "plan_current.yaml"
PLAN_CUR_J  = PROJECT_DIR / "plan_current.json"
BGM_FILE    = PROJECT_DIR / "bgm.mp3"

# ---------------- Templates ----------------
ENV_TEMPLATE = (
    "OPENAI_API_KEY=\n"
    "AZURE_SPEECH_KEY=\n"
    "AZURE_SPEECH_REGION=\n"
    "MINIMAX_SPEECH_KEY=\n"
    "MINIMAX_GROUP_ID=\n"
    "ELEVENLABS_API_KEY=\n"
)

TEXT_TEMPLATE = "請在此輸入影片腳本文字，每行為一個分鏡。\n"

PLAN_SKELETON = {
    "project_title": "DemoVideo",
    "date": datetime.datetime.now().strftime('%Y%m%d'),
    "version": 1,
    "global": {
        "resolution": [1920,1080],
        "fps": 30,
        "loudnorm": {"target_i": -14, "target_tp": -1.5, "target_lra": 11, "mode": "two-pass"},
        "music": {
            "file": "bgm.mp3",
            "gain_db": -12,
            "ducking": {"enabled": True, "threshold": 0.05, "ratio": 8, "attack_ms": 10, "release_ms": 300}
        },
        "subtitle": {
            "font_family": "Noto Sans CJK TC",
            "font_size": 48,
            "outline": 3,
            "shadow": 0,
            "color": "#FFFFFF",
            "align": "bottom-center",
            "margin_v": 60
        },
        "transitions": {"default_type": "crossfade", "default_duration": 0.6}
    },
    "scenes": []
}

# ---------------- Helpers ----------------
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def ensure_file(p: Path, content: str):
    if not p.exists():
        p.write_text(content, encoding="utf-8")
        print(f"[INIT] Created {p.name}")
    else:
        print(f"[OK] {p.name} exists")

def pip_install(pkg: str):
    print(f"[pip] Installing {pkg} ...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

def ensure_packages(package_list):
    installed_status = {}
    for pkg in package_list:
        mod_name = "yaml" if pkg == "pyyaml" else ("PIL" if pkg == "pillow" else pkg)
        try:
            __import__(mod_name)
            installed_status[pkg] = "present"
        except ImportError:
            installed_status[pkg] = "missing"
    to_install = [p for p, st in installed_status.items() if st == "missing"]
    if to_install and not SKIP_INSTALL:
        for p in to_install:
            pip_install(p)
    elif to_install and SKIP_INSTALL:
        print("[SKIP_INSTALL] Missing packages not installed:", to_install)
    # Re-import
    import importlib
    globals()["yaml"] = importlib.import_module("yaml")
    try:
        globals()["PIL_Image"] = importlib.import_module("PIL.Image")
    except Exception:
        globals()["PIL_Image"] = None
    try:
        globals()["dotenv"] = importlib.import_module("dotenv")
    except Exception:
        globals()["dotenv"] = None
    return installed_status

def load_text_lines():
    if not TEXT_FILE.exists():
        return []
    return [l.strip() for l in TEXT_FILE.read_text(encoding="utf-8").splitlines() if l.strip()]

def create_initial_plan():
    import yaml
    PLAN_INIT_Y.write_text(yaml.safe_dump(PLAN_SKELETON, allow_unicode=True), encoding="utf-8")
    PLAN_INIT_J.write_text(json.dumps(PLAN_SKELETON, ensure_ascii=False, indent=2), encoding="utf-8")
    print("[INIT] plan_initial.yaml/json created")

def sync_plan_current():
    import yaml
    if PLAN_CUR_Y.exists():
        try:
            plan = yaml.safe_load(PLAN_CUR_Y.read_text(encoding="utf-8"))
        except Exception:
            plan = None
    else:
        plan = None
    lines = load_text_lines()
    if plan is None:
        if PLAN_INIT_Y.exists():
            try:
                plan = yaml.safe_load(PLAN_INIT_Y.read_text(encoding="utf-8"))
            except Exception:
                plan = PLAN_SKELETON.copy()
        else:
            plan = PLAN_SKELETON.copy()

    existing_map = {sc.get("number"): sc for sc in plan.get("scenes", []) if isinstance(sc, dict)}
    new_scenes = []
    for idx, line in enumerate(lines, start=1):
        prev = existing_map.get(idx, {})
        new_scenes.append({
            "number": idx,
            "raw_text": line,
            "subtitle_text": prev.get("subtitle_text", line),
            "ai_prompt": prev.get("ai_prompt", line),
            "tags": prev.get("tags", []),
            "notes": prev.get("notes", ""),
            "tts": prev.get("tts", {"server":"local","voice":"default","speed":1.0}),
            "transition": prev.get("transition", {
                "type": plan.get("global", {}).get("transitions", {}).get("default_type", "none"),
                "duration": plan.get("global", {}).get("transitions", {}).get("default_duration", 0.0)
            }),
            "flags": prev.get("flags", {"split_subtitle_by_punctuation": True}),
            "hashes": prev.get("hashes", {}),
            "duration_override": prev.get("duration_override"),
            "audio_file": prev.get("audio_file"),
            "image_file": prev.get("image_file"),
        })
    plan["scenes"] = new_scenes
    if "global" not in plan:
        plan["global"] = PLAN_SKELETON["global"]
    plan["date"] = datetime.datetime.now().strftime('%Y%m%d')
    PLAN_CUR_Y.write_text(yaml.safe_dump(plan, allow_unicode=True), encoding="utf-8")
    PLAN_CUR_J.write_text(json.dumps(plan, ensure_ascii=False, indent=2), encoding="utf-8")
    print(f"[OK] plan_current.yaml synced ({len(new_scenes)} scenes).")
    return plan

def load_env():
    if ENV_FILE.exists() and globals().get("dotenv"):
        globals()["dotenv"].load_dotenv(dotenv_path=ENV_FILE)
        return True
    return False

def tool_path(name):
    return shutil.which(name) or "NOT_FOUND"

def key_status(k):
    v = os.environ.get(k, "")
    if not v:
        return "MISSING"
    if len(v.strip()) < 10:
        return "SHORT"
    return "OK"

def list_assets():
    audio = sorted([p.name for p in PROJECT_DIR.glob("*.mp3")] +
                   [p.name for p in PROJECT_DIR.glob("*.wav")] +
                   [p.name for p in PROJECT_DIR.glob("*.m4a")])
    images = []
    for pat in ("*.png","*.jpg","*.jpeg","*.webp"):
        images += [p.name for p in PROJECT_DIR.glob(pat)]
    return audio, sorted(images)

def pillow_test():
    if not PIL_Image or not THUMB_TEST:
        return None
    try:
        test_png = PROJECT_DIR / "pillow_test.png"
        if not test_png.exists():
            img = PIL_Image.new("RGB", (200, 100), color=(30, 120, 200))
            img.save(str(test_png), "PNG")
        # load & verify thumbnail ability
        img2 = PIL_Image.open(str(test_png))
        img2.thumbnail((60, 40))
        return {"file": str(test_png), "size": img2.size}
    except Exception as e:
        return {"error": str(e)}

# ---------------- Execute ----------------
print(f"[INFO] Bootstrapping project: {PROJECT_DIR}")
ensure_dir(PROJECT_DIR)
ensure_dir(ASSETS_DIR)
ensure_dir(FONTS_DIR)

ensure_file(ENV_FILE, ENV_TEMPLATE)
ensure_file(TEXT_FILE, TEXT_TEMPLATE)
ensure_file(BGM_FILE, "")

if not PLAN_INIT_Y.exists():
    create_initial_plan()
else:
    print("[OK] plan_initial.yaml exists")

pkg_status = ensure_packages(REQUIRED_PACKAGES)
env_loaded = load_env()
plan_current = sync_plan_current()

ffmpeg_path  = tool_path("ffmpeg")
ffprobe_path = tool_path("ffprobe")

api_keys = {
    "OPENAI_API_KEY": key_status("OPENAI_API_KEY"),
    "AZURE_SPEECH_KEY": key_status("AZURE_SPEECH_KEY"),
    "AZURE_SPEECH_REGION": key_status("AZURE_SPEECH_REGION"),
    "MINIMAX_SPEECH_KEY": key_status("MINIMAX_SPEECH_KEY"),
    "MINIMAX_GROUP_ID": key_status("MINIMAX_GROUP_ID"),
    "ELEVENLABS_API_KEY": key_status("ELEVENLABS_API_KEY"),
}

audio_files, image_files = list_assets()
thumb_result = pillow_test()

summary = {
    "project_dir": str(PROJECT_DIR),
    "assets_dir": str(ASSETS_DIR),
    "fonts_dir": str(FONTS_DIR),
    "env_file": str(ENV_FILE),
    "text_file": str(TEXT_FILE),
    "plan_initial_yaml": str(PLAN_INIT_Y),
    "plan_current_yaml": str(PLAN_CUR_Y),
    "scenes_in_plan_current": len(plan_current.get("scenes", [])),
    "packages": pkg_status,
    "env_loaded": env_loaded,
    "api_keys_status": api_keys,
    "ffmpeg": ffmpeg_path,
    "ffprobe": ffprobe_path,
    "audio_files_found": len(audio_files),
    "image_files_found": len(image_files),
    "pillow_thumbnail_test": thumb_result,
    "next_steps": [
        "Edit text.txt lines for your scenes (each line = one scene).",
        "Run Step 1 editor (auto-assign UI with thumbnails).",
        "Use Quick Check -> Save YAML.",
        "Run Step 2 renderer."
    ]
}

print("\n--- STEP 0 SUMMARY ---")
print(json.dumps(summary, ensure_ascii=False, indent=2))

if ffmpeg_path == "NOT_FOUND" or ffprobe_path == "NOT_FOUND":
    print("\n⚠️  ffmpeg/ffprobe missing. Install before rendering (e.g. Windows: choco install ffmpeg).")
else:
    print("\n✅ ffmpeg ready.")

missing_keys = [k for k, st in api_keys.items() if st == "MISSING"]
if missing_keys:
    print(f"⚠️  Missing API keys in .env: {', '.join(missing_keys)} (edit .env)")

print("Step 0 complete.")

[INFO] Bootstrapping project: C:\Users\flyre\mynotebooks
[OK] .env exists
[OK] text.txt exists
[OK] bgm.mp3 exists
[OK] plan_initial.yaml exists
[pip] Installing python-dotenv ...
[OK] plan_current.yaml synced (10 scenes).

--- STEP 0 SUMMARY ---
{
  "project_dir": "C:\\Users\\flyre\\mynotebooks",
  "assets_dir": "C:\\Users\\flyre\\mynotebooks\\assets",
  "fonts_dir": "C:\\Users\\flyre\\mynotebooks\\fonts",
  "env_file": "C:\\Users\\flyre\\mynotebooks\\.env",
  "text_file": "C:\\Users\\flyre\\mynotebooks\\text.txt",
  "plan_initial_yaml": "C:\\Users\\flyre\\mynotebooks\\plan_initial.yaml",
  "plan_current_yaml": "C:\\Users\\flyre\\mynotebooks\\plan_current.yaml",
  "scenes_in_plan_current": 10,
  "packages": {
    "pyyaml": "present",
    "pillow": "present",
    "ipywidgets": "present",
    "python-dotenv": "missing"
  },
  "env_loaded": true,
  "api_keys_status": {
    "OPENAI_API_KEY": "OK",
    "AZURE_SPEECH_KEY": "OK",
    "AZURE_SPEECH_REGION": "SHORT",
    "MINIMAX_SPEECH_KEY": 

In [16]:
# 一站式：步驟0（全域設定：TTS / 字幕比例 / 後製 + 內建飄落：簡易/進階 + 一鍵檔位 省資源/均衡/頂規）
#        步驟1（分鏡編輯）→ 步驟2（輸出）
# 檔位說明：
# - 省資源：720p/25fps、關閉重濾鏡、內建(簡易)飄落、進階禁用、CPU 最省
# - 均衡：1080p/25fps、內建(進階)半尺寸生成（疊時放大）、VP9 快速參數
# - 頂規：1080p/25fps（可自行改 4K）、內建(進階)全尺寸、VP9 較高品質參數（慢，但最自然）
#
# 使用方式：
# 1) 上一格設定：main_dir = r"C:/Users/you/yourfolder"
# 2) 執行本格 → 步驟0：點「一鍵檔位」(省資源/均衡/頂規) → 看面板參數有變 → 按「儲存設定」
# 3) 步驟1：載入文字(重建) → 逐場景補音訊/圖片（TTS/AI生圖）→ 可按「儲存規劃檔」
# 4) 按「輸出影片」，會自動用檔位覆蓋解析度/幀率並套特效；透明特效首次會生成，再者重用

import os, re, json, yaml, subprocess, shutil, datetime, hashlib, textwrap, base64, math, random, tempfile
from pathlib import Path
from IPython.display import display, clear_output
import ipywidgets as W

# ---------------- 基本環境 ----------------
try:
    main_dir
except NameError:
    main_dir = str(Path.cwd())
main_dir = Path(main_dir)
if not main_dir.exists():
    raise RuntimeError(f"主目錄不存在: {main_dir}")
print(f"[初始化] 主目錄 = {main_dir}")

PLAN_CURRENT   = "plan_current.yaml"
HANDSHAKE_NAME = "plan_handshake.json"
LOCK_NAME      = "plan_lock.json"
TEXT_FILE_NAME = "text.txt"
SETTINGS_NAME  = "settings.json"

DEFAULT_FALLBACK_DURATION = 3.0
LOUDNORM_I   = -16.0
LOUDNORM_TP  = -1.0
LOUDNORM_LRA = 11.0
FONTNAME_FORCE_STYLE = "Arial"

# 環境變數（若使用雲端 TTS 需設定）
OPENAI_API_KEY  = os.environ.get("OPENAI_API_KEY","").strip()
OPENAI_ORG_ID   = os.environ.get("OPENAI_ORG_ID","").strip()
AZURE_TTS_KEY   = os.environ.get("AZURE_TTS_KEY","").strip()
AZURE_TTS_REGION= os.environ.get("AZURE_TTS_REGION","").strip()
MINIMAX_API_KEY = os.environ.get("MINIMAX_API_KEY","").strip()

import subprocess
from pathlib import Path
import shutil

def convert_webm_overlay_to_pngs(webm_path, png_prefix, frame_rate=25):
    """
    將 webm 特效檔解出 png 幀圖，回傳 png 路徑 pattern字串與目錄
    """
    webm_path = Path(webm_path)
    png_dir = webm_path.parent / "_overlaytemp"
    png_dir.mkdir(exist_ok=True)
    png_pattern = str(png_dir / f"{png_prefix}_%06d.png")
    for f in png_dir.glob(f"{png_prefix}_*.png"):
        f.unlink()
    cmd = [
        "ffmpeg", "-y", "-i", str(webm_path),
        "-vf", "pad=ceil(iw/2)*2:ceil(ih/2)*2",
        png_pattern
    ]
    subprocess.run(cmd, check=True)
    return png_pattern, png_dir

def cleanup_overlay_pngs(png_dir, png_prefix):
    for f in png_dir.glob(f"{png_prefix}_*.png"):
        f.unlink()
    try:
        png_dir.rmdir()
    except Exception:
        pass

def postprocess_final_video(base_plain: Path, settings: dict, out_path: Path, base_w=1920, base_h=1080, base_fps=25):
    post = (settings.get("post") or {})
    nothing = True
    if (post.get("color") or {}).get("enabled", False): nothing=False
    if (post.get("denoise") or {}).get("enabled", False): nothing=False
    if (post.get("sharpen") or {}).get("enabled", False): nothing=False
    if (post.get("fps_interp") or {}).get("enabled", False): nothing=False
    for ov in (post.get("overlays") or []):
        if ov.get("enabled", False): nothing=False
    if nothing:
        shutil.copyfile(base_plain, out_path); return True, "copy"

    extra_inputs, fc, vout = build_filter_chain_and_inputs(base_w, base_h, settings, base_fps=base_fps)

    # --- 新增判斷：若唯一 overlay 為 .webm，則轉成 png 疊圖 ---
    # 只補單一 webm overlay（如有多層可再遞迴；現在補你特效全黑問題）    # --- 新：跨option安全尋找 webm overlay 檔案 ---
    webm_overlay = None
    # extra_inputs 常是：['-stream_loop', '-1', '-i', 'xxx.webm', ...]
    for i in range(len(extra_inputs) - 1):
        if (
            isinstance(extra_inputs[i], str)
            and extra_inputs[i] == "-i"
            and isinstance(extra_inputs[i + 1], str)
            and extra_inputs[i + 1].lower().endswith(".webm")
        ):
            webm_overlay = Path(extra_inputs[i + 1])
            break
    # 備援：暴力掃 .webm（萬一 pattern 變動也不會失效）
    if not webm_overlay:
        for item in extra_inputs:
            if isinstance(item, str) and item.lower().endswith(".webm"):
                webm_overlay = Path(item)
                break

    if webm_overlay and "overlay" in fc:
        # (1) 先拆 webm 成 png
        png_pattern, png_dir = convert_webm_overlay_to_pngs(webm_overlay, "adv_overlay", frame_rate=base_fps)
        # (2) 用 png 幀序列疊圖
        cmd = [
            "ffmpeg", "-y", "-i", str(base_plain),
            "-framerate", str(base_fps), "-i", png_pattern,
            "-filter_complex", "[0:v][1:v]overlay=0:0:shortest=1[v1]",
            "-map", "[v1]", "-map", "0:a?", "-c:v", "libx264", "-pix_fmt", "yuv420p", "-c:a", "aac", "-movflags", "+faststart", str(out_path)
        ]
        res = subprocess.run(cmd, capture_output=True)
        err = res.stderr.decode("utf-8", errors="ignore") if res.stderr else ""        
        cleanup_overlay_pngs(png_dir, "adv_overlay")
        ok = (res.returncode==0 and out_path.exists() and out_path.stat().st_size>0)
        return ok, (res.stderr or "")[-400:] if not ok else "ok"
    else:
        # 沒有 webm overlay, 走原本路徑
        cmd = ["ffmpeg","-y","-i",str(base_plain)] + extra_inputs + [
            "-filter_complex", fc, "-map", f"[{vout}]", "-map", "0:a?","-c:v","libx264","-pix_fmt","yuv420p","-c:a","aac","-movflags","+faststart", str(out_path)
        ]
        res=_run(cmd, capture=True)
        ok = (res.returncode==0 and out_path.exists() and out_path.stat().st_size>0)
        return ok, (res.stderr or "")[-400:] if not ok else "ok"

# ---------------- 常用工具 ----------------
def _run(cmd, capture=False):
    cmd = [str(c) for c in cmd]
    return subprocess.run(cmd,
                          stdout=(subprocess.PIPE if capture else None),
                          stderr=(subprocess.PIPE if capture else None),
                          text=True)

def _md5_text(txt: str):
    return hashlib.md5(txt.encode("utf-8")).hexdigest()

def _ffprobe_duration(path: Path):
    if not shutil.which("ffprobe"):
        return None
    r = subprocess.run(["ffprobe","-v","error","-show_entries","format=duration",
                        "-of","default=noprint_wrappers=1:nokey=1", str(path)],
                       stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    try:
        return max(0.1, float((r.stdout or "").strip()))
    except:
        return None

def atomic_write_text(path: Path, text: str):
    tmp = path.with_suffix(path.suffix + ".tmp")
    tmp.write_text(text, encoding="utf-8")
    tmp.replace(path)

def gather_images():
    exts = {".png",".jpg",".jpeg",".webp"}
    return [""] + [p.name for p in sorted(main_dir.iterdir()) if p.suffix.lower() in exts]

def gather_audios():
    exts = {".mp3",".wav",".m4a",".aac",".flac",".ogg"}
    return [""] + [p.name for p in sorted(main_dir.iterdir()) if p.suffix.lower() in exts]

def gather_overlay_candidates():
    exts = {".mp4",".mov",".webm",".mkv",".avi"}
    return [""] + [p.name for p in sorted(main_dir.iterdir()) if p.suffix.lower() in exts]

def ensure_text_lines():
    txt = main_dir / TEXT_FILE_NAME
    if not txt.exists():
        return []
    return [l.strip() for l in txt.read_text(encoding="utf-8").splitlines() if l.strip()]

# ---------------- 設定（讀寫 + 預設 + 檔位） ----------------
def default_settings():
    return {
        "tts": {
            "provider": "OPENAI",
            "openai": {"model":"tts-1","voice":"alloy","format":"mp3"},
            "azure":  {"region":"", "voice":"zh-TW-HsiaoChenNeural", "rate_pct":0, "pitch_pct":0, "format":"audio-24khz-48kbitrate-mono-mp3"},
            "minimax":{"model":"speech-01","voice_id":"female-1","format":"mp3"}
        },
        "subs": {"scale_pct": 125},
        "profile": {
            "name": "eco",                 # eco | balanced | ultra
            "output": {"w":1280,"h":720,"fps":25},
            "adv_half_res": True,          # 進階粒子素材半尺寸生成(疊時放大)
            "vp9": {"crf":32, "cpu_used":8, "tile_columns":2, "row_mt":1, "auto_alt_ref":0}
        },
        "post": {
            "denoise": {"enabled": False, "preset":"medium"},
            "sharpen": {"enabled": False, "amount": 0.8},
            "color":   {"enabled": False, "brightness":0.0, "contrast":1.0, "saturation":1.0},
            "fps_interp": {"enabled": False, "fps": 60},
            "overlays": [
                {
                    "enabled": True,
                    "mode": "builtin_simple",   # file | builtin_simple | builtin_advanced
                    "file": "",
                    "type": "alpha",
                    "opacity": 1.0,
                    "scale_to_output": True,
                    "black": {"mode":"screen"},
                    "greenscreen":{"color":"0x00FF00","similarity":0.20,"blend":0.10},
                    "simple":   {"kind":"snow","density":35,"size":36,"speed":130,"color":"white"},
                    "advanced": {"kind":"snow","density":60,"size_min":24,"size_max":64,"speed_min":60,"speed_max":180,
                                 "drift":60,"turbulence":25,"rotation_min":-60,"rotation_max":60,
                                 "layers":{"near":0.4,"mid":0.35,"far":0.25},"loop_sec":8,"seed":1337,"texture_file":""}
                },
                {
                    "enabled": False,
                    "mode": "builtin_simple",
                    "file": "",
                    "type": "alpha",
                    "opacity": 0.9,
                    "scale_to_output": True,
                    "black": {"mode":"screen"},
                    "greenscreen":{"color":"0x00FF00","similarity":0.20,"blend":0.10},
                    "simple":   {"kind":"leaves","density":20,"size":48,"speed":160,"color":"white"},
                    "advanced": {"kind":"leaves","density":36,"size_min":24,"size_max":72,"speed_min":80,"speed_max":160,
                                 "drift":90,"turbulence":35,"rotation_min":-120,"rotation_max":120,
                                 "layers":{"near":0.5,"mid":0.3,"far":0.2},"loop_sec":8,"seed":2025,"texture_file":""}
                }
            ]
        }
    }

def settings_path():
    return main_dir/SETTINGS_NAME

def load_settings():
    sp = settings_path()
    if sp.exists():
        try:
            s = json.loads(sp.read_text(encoding="utf-8"))
            # 合併新欄位預設
            def merge_defaults(d, tmpl):
                for k,v in tmpl.items():
                    if k not in d:
                        d[k]=v
                    else:
                        if isinstance(v, dict) and isinstance(d[k], dict):
                            merge_defaults(d[k], v)
                return d
            s = merge_defaults(s, default_settings())
            return s
        except:
            pass
    s = default_settings()
    save_settings(s)
    return s

def save_settings(s):
    settings_path().write_text(json.dumps(s, ensure_ascii=False, indent=2), encoding="utf-8")

# ---------------- Plan 建立 ----------------
def build_plan_from_lines(lines):
    scenes=[]
    for i,line in enumerate(lines, start=1):
        scenes.append({
            "number": i,
            "subtitle_text": line,
            "raw_text": line,
            "ai_prompt": line,
            "audio_file": None,
            "image_file": None,
            "flags": {"enable_ducking": True},
            "tts": {"voice":"alloy","language":"zh-TW"},
            "transition": {"type":"none","duration":0.0},
            "duration_override": None
        })
    plan = {
        "project_title": "OneStepYamlEditor",
        "date": datetime.datetime.now().strftime("%Y%m%d"),
        "version": 1,
        "global": {
            "resolution":[1920,1080],
            "fps":25,
            "subtitle":{
                "font_family":"Arial","font_size":48,"outline":3,"shadow":0,
                "color":"#FFFFFF","align":"bottom-center","margin_v":60
            },
            "loudnorm":{"target_i":LOUDNORM_I,"target_tp":LOUDNORM_TP,"target_lra":LOUDNORM_LRA,"mode":"two-pass"},
            "music":{"file":None,"gain_db":-8,"ducking":{"threshold":0.1,"ratio":8.0,"attack_ms":10,"release_ms":250}},
            "transitions":{"default_type":"none","default_duration":0.0}
        },
        "scenes": scenes
    }
    return plan

def plan_scene_count(plan):
    sc = plan.get("scenes",[])
    return len(sc) if isinstance(sc, list) else 0

def write_handshake_lock(plan_yaml_text: str, scenes_count: int):
    md5 = _md5_text(plan_yaml_text)
    hs = {"filename": PLAN_CURRENT, "scenes": scenes_count, "md5": md5, "saved_at": datetime.datetime.now().isoformat(timespec="seconds")}
    atomic_write_text(main_dir/HANDSHAKE_NAME, json.dumps(hs, ensure_ascii=False, indent=2))
    lk = {"filename": PLAN_CURRENT, "locked_at": datetime.datetime.now().isoformat(timespec="seconds")}
    atomic_write_text(main_dir/LOCK_NAME, json.dumps(lk, ensure_ascii=False, indent=2))
    return md5

def backup_plan_yaml(plan_text: str):
    stamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    (main_dir / f"plan_{stamp}.yaml").write_text(plan_text, encoding="utf-8")
    return f"plan_{stamp}.yaml"

# ---------------- 字幕工具 ----------------
def _hex_to_ass_color(hx: str):
    try:
        s=hx.strip()
        if s.startswith("#"): s=s[1:]
        r=int(s[0:2],16); g=int(s[2:4],16); b=int(s[4:6],16)
        return f"&H{b:02X}{g:02X}{r:02X}&"
    except:
        return "&HFFFFFF&"

def _seconds_to_ass(sec: float):
    sec=max(0.0, float(sec))
    h=int(sec//3600); m=int((sec%3600)//60); s=int(sec%60); cs=int(round((sec-int(sec))*100))
    return f"{h}:{m:02d}:{s:02d}.{cs:02d}"

def _seconds_to_srt(sec: float):
    sec=max(0.0, float(sec))
    h=int(sec//3600); m=int((sec%3600)//60); s=int(sec%60); ms=int(round((sec-int(sec))*1000))
    return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"

def build_subtitle_events(plan):
    events=[]; t=0.0
    for sc in plan.get("scenes",[]):
        text=sc.get("subtitle_text") or sc.get("raw_text") or ""
        dur=None
        af=sc.get("audio_file")
        if af and (main_dir/af).exists():
            dur=_ffprobe_duration(main_dir/af)
        if dur is None:
            dur=sc.get("duration_override") or DEFAULT_FALLBACK_DURATION
        start=t; end=t+dur
        events.append({"scene":sc.get("number",0),"start":start,"end":end,"text":text})
        t=end
    return events

def write_ass(events, plan, out_path=None, settings=None):
    sub_cfg=plan.get("global",{}).get("subtitle",{}) or {}
    font=sub_cfg.get("font_family","Arial"); size=int(sub_cfg.get("font_size",48))
    outline=int(sub_cfg.get("outline",3)); shadow=int(sub_cfg.get("shadow",0))
    color=_hex_to_ass_color(sub_cfg.get("color","#FFFFFF"))
    align_map={'bottom-left':1,'bottom-center':2,'bottom-right':3,'middle-left':4,'middle-center':5,'middle-right':6,'top-left':7,'top-center':8,'top-right':9}
    align=align_map.get(sub_cfg.get("align","bottom-center"),2)
    margin_v=int(sub_cfg.get("margin_v",60))
    w,h=plan.get("global",{}).get("resolution",[1920,1080])
    scale_pct = 125
    if settings and isinstance(settings, dict):
        scale_pct = int((settings.get("subs") or {}).get("scale_pct", scale_pct))
    scale_x=scale_pct; scale_y=scale_pct

    header=textwrap.dedent(f"""\
    [Script Info]
    PlayResX: {w}
    PlayResY: {h}
    ScaledBorderAndShadow: yes

    [V4+ Styles]
    Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
    Style: Default,{font},{size},{color},&H000000FF,&H00000000,&H00000000,0,0,0,0,{scale_x},{scale_y},0,0,1,{outline},{shadow},{align},30,30,{margin_v},1

    [Events]
    Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
    """)
    lines=[header]
    for e in events:
        start_str=_seconds_to_ass(e['start']); end_str=_seconds_to_ass(e['end'])
        text_raw=e.get('text') or ''
        text_ass=text_raw.replace('\n', r'\N')
        lines.append(f"Dialogue: 0,{start_str},{end_str},Default,,0,0,0,,{text_ass}\n")
    if out_path is None:
        out_path = main_dir / "combined.ass"
    else:
        out_path = Path(out_path)
    out_path.write_text(''.join(lines),encoding="utf-8")
    return out_path

def write_srt(events, out_path: Path):
    out=[]
    for i,e in enumerate(events, start=1):
        start_str=_seconds_to_srt(e['start']); end_str=_seconds_to_srt(e['end'])
        text_raw=e.get('text') or ''
        out.append(f"{i}\n{start_str} --> {end_str}\n{text_raw}\n\n")
    out_path.write_text('.'.join(out).replace('..','.'),encoding="utf-8") if False else out_path.write_text(''.join(out),encoding="utf-8")
    return out_path

# ---------------- 音訊正規化 ----------------
def validate_audio(audio_path: Path, fallback_sec=3.0) -> Path:
    audio_path=Path(audio_path)
    if not audio_path.exists() or audio_path.stat().st_size==0:
        sil=audio_path.with_suffix(".silence.wav")
        _run(["ffmpeg","-y","-f","lavfi","-i","anullsrc=r=48000:cl=stereo","-t",f"{fallback_sec:.3f}",
              "-c:a","pcm_s16le", str(sil)])
        return sil
    res=_run(["ffmpeg","-v","error","-i",str(audio_path),"-f","null","-"], capture=True)
    if res.returncode!=0:
        sil=audio_path.with_suffix(".silence.wav")
        _run(["ffmpeg","-y","-f","lavfi","-i","anullsrc=r=48000:cl=stereo","-t",f"{fallback_sec:.3f}",
              "-c:a","pcm_s16le", str(sil)])
        return sil
    return audio_path

def loudnorm_two_pass(in_path: Path, out_path: Path) -> bool:
    probe=_run(["ffmpeg","-i",str(in_path),"-af",
                f"loudnorm=I={LOUDNORM_I}:TP={LOUDNORM_TP}:LRA={LOUDNORM_LRA}:print_format=json",
                "-f","null","-"], capture=True)
    blob=(probe.stdout or "")+(probe.stderr or "")
    m=re.search(r'\{[\s\S]*?\}', blob)
    if not m: return False
    try:
        data=json.loads(m.group(0))
        filt=(f"loudnorm=I={LOUDNORM_I}:TP={LOUDNORM_TP}:LRA={LOUDNORM_LRA}:"
              f"input_i={data['input_i']}:input_tp={data['input_tp']}:input_lra={data['input_lra']}:"
              f"input_thresh={data['input_thresh']}:offset={data['target_offset']}:linear=true:print_format=summary")
        res=_run(["ffmpeg","-y","-i",str(in_path),"-af",filt,str(out_path)], capture=True)
        return res.returncode==0 and out_path.exists() and out_path.stat().st_size>0
    except:
        return False

def loudnorm_one_pass(in_path: Path, out_path: Path) -> bool:
    res=_run(["ffmpeg","-y","-i",str(in_path),"-af",
              f"loudnorm=I={LOUDNORM_I}:TP={LOUDNORM_TP}:LRA={LOUDNORM_LRA}", str(out_path)], capture=True)
    return res.returncode==0 and out_path.exists() and out_path.stat().st_size>0

def process_scene_audio_return_final(sc, fallback_sec=DEFAULT_FALLBACK_DURATION) -> Path:
    num=sc.get("number"); af=sc.get("audio_file")
    if not af or not (main_dir/af).exists():
        silent=main_dir/f"scene_{num:03d}_silent.wav"
        _run(["ffmpeg","-y","-f","lavfi","-i","anullsrc=r=48000:cl=stereo","-t",f"{fallback_sec:.3f}",
              "-c:a","pcm_s16le", str(silent)])
        af=silent.name; sc["audio_file"]=af
    src=validate_audio(main_dir/af, fallback_sec=fallback_sec)
    norm=main_dir/f"scene_{num:03d}_norm.wav"
    ok=loudnorm_two_pass(src, norm)
    if not ok: ok=loudnorm_one_pass(src, norm)
    final_audio=norm if ok else src
    return validate_audio(final_audio, fallback_sec=fallback_sec)

# ---------------- TTS ----------------
def tts_generate(text: str, out_path: Path, settings: dict, default_voice="alloy"):
    prov = (settings.get("tts") or {}).get("provider","OPENAI")
    if prov=="OPENAI":
        o = (settings["tts"]).get("openai",{})
        voice=o.get("voice", default_voice)
        model=o.get("model","tts-1")
        fmt=o.get("format","mp3")
        return tts_openai(text, out_path.with_suffix(f".{fmt}"), voice=voice, model=model)
    elif prov=="AZURE":
        ax = (settings["tts"]).get("azure",{})
        voice=ax.get("voice","zh-TW-HsiaoChenNeural")
        fmt=ax.get("format","audio-24khz-48kbitrate-mono-mp3")
        rate_pct=int(ax.get("rate_pct",0)); pitch_pct=int(ax.get("pitch_pct",0))
        region=ax.get("region") or AZURE_TTS_REGION
        return tts_azure(text, out_path.with_suffix(".mp3"), voice=voice, rate_pct=rate_pct, pitch_pct=pitch_pct, fmt=fmt, region=region)
    elif prov=="MINIMAX":
        mm = (settings["tts"]).get("minimax",{})
        model=mm.get("model","speech-01"); voice_id=mm.get("voice_id","female-1")
        fmt=mm.get("format","mp3")
        return tts_minimax(text, out_path.with_suffix(f".{fmt}"), model=model, voice_id=voice_id)
    else:
        return False, f"未知 TTS 供應商: {prov}"

def tts_openai(text: str, out_path: Path, voice="alloy", model="tts-1"):
    if not OPENAI_API_KEY:
        return False,"缺少 OPENAI_API_KEY"
    import requests
    try:
        url="https://api.openai.com/v1/audio/speech"
        headers={"Authorization":f"Bearer {OPENAI_API_KEY}","Content-Type":"application/json"}
        if OPENAI_ORG_ID: headers["OpenAI-Organization"]=OPENAI_ORG_ID
        payload={"model":model,"voice":voice,"input":text or "","response_format":out_path.suffix[1:] or "mp3"}
        r=requests.post(url,headers=headers,json=payload,timeout=180)
        if r.status_code!=200:
            return False,f"{r.status_code} {r.text[:200]}"
        out_path.write_bytes(r.content)
        return True,"OK"
    except Exception as e:
        return False,f"Exception {e}"

def tts_azure(text: str, out_path: Path, voice="zh-TW-HsiaoChenNeural", rate_pct=0, pitch_pct=0, fmt="audio-24khz-48kbitrate-mono-mp3", region=""):
    if not AZURE_TTS_KEY or not (region or AZURE_TTS_REGION):
        return False,"缺少 AZURE_TTS_KEY / AZURE_TTS_REGION"
    import requests
    try:
        endpoint=f"https://{region}.tts.speech.microsoft.com/cognitiveservices/v1"
        ssml=f"""<speak version='1.0' xml:lang='zh-TW'>
  <voice name='{voice}'><prosody rate='{rate_pct:+d}%' pitch='{pitch_pct:+d}%'>
    {text or ""}
  </prosody></voice>
</speak>"""
        headers={
            "Ocp-Apim-Subscription-Key": AZURE_TTS_KEY,
            "Content-Type": "application/ssml+xml",
            "X-Microsoft-OutputFormat": fmt,
            "User-Agent": "OneStepYamlEditor"
        }
        r=requests.post(endpoint, headers=headers, data=ssml.encode("utf-8"), timeout=180)
        if r.status_code!=200:
            return False,f"{r.status_code} {r.text[:200]}"
        out_path.write_bytes(r.content)
        return True,"OK"
    except Exception as e:
        return False,f"Exception {e}"

def tts_minimax(text: str, out_path: Path, model="speech-01", voice_id="female-1"):
    if not MINIMAX_API_KEY:
        return False,"缺少 MINIMAX_API_KEY"
    return False,"Minimax TTS 尚未實作（請提供 API 規格）"

# ---------------- 片段組合 ----------------
def make_scene_clip(image_file, audio_file, duration, w, h, fps, out_path: Path):
    """Create scene clip ensuring image/video duration strictly matches audio duration to prevent black frames."""
    out_path=Path(out_path)
    audio_p=main_dir/audio_file if isinstance(audio_file,str) else Path(audio_file)
    img_p=(main_dir/image_file) if (image_file and isinstance(image_file,str)) else None
    
    # Verify actual audio duration for strict matching
    actual_duration = duration
    if audio_p and audio_p.exists():
        probe_dur = _ffprobe_duration(audio_p)
        if probe_dur and probe_dur > 0.1:
            actual_duration = probe_dur

    def cmd_aac(shortest=True, use_color=False):
        # Use -shortest AND explicit -t to ensure strict duration matching and prevent black frames
        base=(["-f","lavfi","-t",f"{actual_duration:.3f}","-i",f"color=black:s={w}x{h}:r={fps}"] if use_color
              else ["-loop","1","-framerate",str(fps),"-t",f"{actual_duration:.3f}","-i",str(img_p)])
        return (["ffmpeg","-y"]+base+["-i",str(audio_p)]+
                ["-t",f"{actual_duration:.3f}","-shortest"]+  # Always use both for strict sync
                (["-filter:v",f"scale={w}:{h}:force_original_aspect_ratio=decrease,pad={w}:{h}:(ow-iw)/2:(oh-ih)/2"] if not use_color else [])+
                ["-r",str(fps),"-c:v","libx264","-pix_fmt","yuv420p","-c:a","aac","-b:a","192k","-movflags","+faststart",str(out_path)])

    def cmd_mp3(use_color=False):
        # Add -shortest to ensure video doesn't extend beyond audio
        base=(["-f","lavfi","-i",f"color=black:s={w}x{h}:r={fps}"] if use_color
              else ["-loop","1","-framerate",str(fps),"-i",str(img_p)])
        return (["ffmpeg","-y"]+base+["-i",str(audio_p),"-t",f"{actual_duration:.3f}","-shortest"]+
                (["-filter:v",f"scale={w}:{h}:force_original_aspect_ratio=decrease,pad={w}:{h}:(ow-iw)/2:(oh-ih)/2"] if not use_color else [])+
                ["-r",str(fps),"-c:v","libx264","-pix_fmt","yuv420p","-c:a","libmp3lame","-q:a","3","-movflags","+faststart",str(out_path)])

    def cmd_last():
        return ["ffmpeg","-y","-f","lavfi","-t",f"{duration:.3f}","-i",f"color=black:s={w}x{h}:r={fps}",
                "-f","lavfi","-t",f"{duration:.3f}","-i","anullsrc=r=48000:cl=stereo",
                "-r",str(fps),"-c:v","libx264","-pix_fmt","yuv420p","-c:a","aac","-b:a","192k","-movflags","+faststart",str(out_path)]

    attempts=[]
    if img_p and img_p.exists():
        attempts.append(("A_aac_shortest", cmd_aac(shortest=True, use_color=False)))
        attempts.append(("B_aac_fixed_t",  cmd_aac(shortest=False, use_color=False)))
        attempts.append(("C_mp3_fixed_t",  cmd_mp3(use_color=False)))
    attempts.append(("D_mp3_color_t",     cmd_mp3(use_color=True)))
    attempts.append(("E_synthetic",       cmd_last()))

    for label,cmd in attempts:
        res=_run(cmd, capture=True)
        if res.returncode==0 and out_path.exists() and out_path.stat().st_size>0:
            return True,label
    return False,"failed"

# ---------------- 內建(簡易) 飄落（drawtext） ----------------
def builtin_simple_node(kind: str, W: int, H: int, fps: int, density: int, size_px: int, speed: int, color: str, layer_index: int, opacity: float = 1.0):
    glyph = "❄" if kind=="snow" else ("│" if kind=="rain" else "🍁")
    sway_amp = 40 if kind=="snow" else (8 if kind=="rain" else 80)
    nodes = [f"color=c=black@0.0:s={W}x{H}:r={fps},format=rgba"]
    n_items = max(1, int(density))
    spd = max(1, int(speed))
    a = max(0.0, min(1.0, float(opacity)))
    for i in range(n_items):
        xi = f"(w/{n_items})*{i}+{sway_amp}*sin(t*0.8+{i})"
        yi = f"mod(t*{spd} + (h/{n_items})*{i}, h)"
        nodes.append(
            f"drawtext=text='{glyph}':fontcolor={color}:fontsize={int(size_px)}:x='{xi}':y='{yi}':alpha={a}"
        )
    out_label = f"ovb{layer_index}"
    graph = ",".join(nodes) + f"[{out_label}]"
    return graph, out_label

# ---------------- 內建(進階) 透明特效（生成透明 VP9 WebM並快取） ----------------
def _write_default_texture(kind: str) -> Path:
    # 用 PIL 生成簡單圖樣（免外部檔）
    from PIL import Image, ImageDraw
    p = main_dir / f"_adv_tex_{kind}.png"
    if p.exists():
        return p
    img = Image.new("RGBA",(128,128),(255,255,255,0))
    d = ImageDraw.Draw(img)
    if kind=="snow":
        # 簡單六角雪花
        cx, cy, r = 64, 64, 40
        for ang in range(0,360,60):
            x = cx + r*math.cos(math.radians(ang))
            y = cy + r*math.sin(math.radians(ang))
            d.line([(cx,cy),(x,y)], fill=(255,255,255,220), width=6)
    elif kind=="rain":
        d.line([(64,8),(96,120)], fill=(255,255,255,220), width=10)
    else:
        # 葉片菱形
        d.polygon([(64,8),(120,64),(64,120),(8,64)], fill=(255,180,0,220))
    img.save(p)
    return p

def _ensure_adv_overlay_clip(layer_idx: int, adv_cfg: dict, base_w: int, base_h: int, fps: int, opacity: float, profile: dict) -> (Path, tuple):
    """
    依設定生成/重用透明 WebM，回傳 (clip_path, (gen_w, gen_h))
    若 profile['adv_half_res'] 為 True，則以半尺寸生成（疊時會放大）
    VP9 參數取自 profile['vp9']
    """
    half = bool(profile.get("adv_half_res", True))
    gen_w = max(2, base_w//2) if half else base_w
    gen_h = max(2, base_h//2) if half else base_h

    vp9 = profile.get("vp9", {"crf":32,"cpu_used":8,"tile_columns":2,"row_mt":1,"auto_alt_ref":0})
    crf = int(vp9.get("crf",32)); cpu = int(vp9.get("cpu_used",8))
    tiles = int(vp9.get("tile_columns",2)); rowmt = int(vp9.get("row_mt",1)); aarf = int(vp9.get("auto_alt_ref",0))

    # 參數雜湊（生成快取）
    key = {"w":gen_w,"h":gen_h,"fps":fps,"adv":adv_cfg,"opa":round(float(opacity),3),"vp9":vp9,"ver":"1.1"}
    hid = _md5_text(json.dumps(key, sort_keys=True))
    out_path = main_dir / f"_adv_overlay_L{layer_idx}_{hid[:10]}.webm"
    meta_path= out_path.with_suffix(".json")
    if out_path.exists() and meta_path.exists():
        try:
            m = json.loads(meta_path.read_text(encoding="utf-8"))
            if m.get("hash")==hid:
                return out_path, (gen_w, gen_h)
        except:
            pass

    # 準備材質
    kind = (adv_cfg.get("kind") or "snow").lower()
    tex_file = adv_cfg.get("texture_file") or ""
    tex_path = Path(tex_file) if tex_file else _write_default_texture("snow" if kind=="snow" else ("rain" if kind=="rain" else "leaves"))
    if not tex_path.exists():
        tex_path = _write_default_texture("snow" if kind=="snow" else ("rain" if kind=="rain" else "leaves"))

    # 生成逐幀 PNG
    loop_sec = max(2, int(adv_cfg.get("loop_sec",8)))
    frames = loop_sec * fps
    from PIL import Image, ImageDraw
    rnd = random.Random(int(adv_cfg.get("seed",1337)))
    N = max(1, int(adv_cfg.get("density",60)))
    size_min = max(6, int(adv_cfg.get("size_min",20)))
    size_max = max(size_min+1, int(adv_cfg.get("size_max",64)))
    spd_min  = max(10, int(adv_cfg.get("speed_min",60)))
    spd_max  = max(spd_min+1, int(adv_cfg.get("speed_max",180)))
    drift    = float(adv_cfg.get("drift",60))
    turb     = float(adv_cfg.get("turbulence",25))
    rot_min  = float(adv_cfg.get("rotation_min",-90))
    rot_max  = float(adv_cfg.get("rotation_max", 90))
    layer_split = adv_cfg.get("layers", {"near":0.4,"mid":0.35,"far":0.25})
    split = [("near", float(layer_split.get("near",0.4))),
             ("mid",  float(layer_split.get("mid",0.35))),
             ("far",  float(layer_split.get("far",0.25)))]
    ssum = sum(v for _,v in split) or 1.0
    split = [(k, v/ssum) for k,v in split]
    depth_mul = {"near":1.15, "mid":1.0, "far":0.85}
    speed_mul = {"near":1.25, "mid":1.0, "far":0.75}
    drift_mul = {"near":1.0,  "mid":0.8, "far":0.6}

    tex_img = Image.open(tex_path).convert("RGBA") if kind!="rain" else None

    particles=[]
    for i in range(N):
        r = rnd.random()
        acc=0.0; group="far"
        for k,p in split:
            acc+=p
            if r<=acc:
                group=k; break
        sx = rnd.uniform(0, gen_w)
        sy = rnd.uniform(0, gen_h)
        sz = rnd.randint(size_min, size_max) * depth_mul[group]
        sp = rnd.uniform(spd_min, spd_max) * speed_mul[group]
        dr = drift * drift_mul[group]
        rs = rnd.uniform(rot_min, rot_max)  # deg/sec
        ph = rnd.uniform(0, math.tau)
        tb = rnd.uniform(0.3,1.2) * turb
        particles.append({"x":sx,"y":sy,"size":sz,"speed":sp,"drift":dr,"rotspd":rs,"phase":ph,"turb":tb,"group":group,"tex":tex_img})

    tmp_dir = Path(tempfile.mkdtemp(prefix=f"advL{layer_idx}_", dir=str(main_dir)))
    try:
        for fi in range(frames):
            t = fi / fps
            canvas = Image.new("RGBA",(gen_w,gen_h),(0,0,0,0))
            draw = ImageDraw.Draw(canvas)
            for p in particles:
                p["y"] = (p["y"] + p["speed"]/fps) % gen_h
                x = (p["x"] + p["drift"]*math.sin(0.8*t + p["phase"]) +
                     p["turb"]*math.sin(1.7*t + p["phase"]*1.23))
                y = p["y"]
                if kind=="rain":
                    length = max(8, int(p["size"]*1.5))
                    dx = int(p["drift"]*0.15)
                    draw.line([(x, y), (x+dx, y+length)], fill=(255,255,255,int(255*opacity)), width=max(1, int(p["size"]*0.08)))
                else:
                    tex = p["tex"]
                    sz  = max(6, int(p["size"]))
                    stamp = tex.resize((sz,sz), Image.LANCZOS)
                    angle = (p["rotspd"]*t) % 360.0
                    stamp_r = stamp.rotate(angle, resample=Image.BICUBIC, expand=True)
                    if opacity < 1.0:
                        a_chan = stamp_r.split()[3]
                        a_chan = a_chan.point(lambda v: int(v*opacity))
                        stamp_r.putalpha(a_chan)
                    cx = int(x - stamp_r.size[0]//2)
                    cy = int(y - stamp_r.size[1]//2)
                    canvas.alpha_composite(stamp_r, (cx, cy))
            canvas.save(tmp_dir / f"f_{fi:06d}.png", "PNG")

        # 透明 VP9
        cmd = ["ffmpeg","-y","-framerate",str(fps),"-i",str(tmp_dir/"f_%06d.png"),
               "-c:v","libvpx-vp9","-pix_fmt","yuva420p","-b:v","0","-crf",str(crf),
               "-cpu-used",str(cpu),"-row-mt",str(rowmt),"-tile-columns",str(tiles),
               "-auto-alt-ref",str(aarf),
               str(out_path)]
        r = _run(cmd, capture=True)
        if r.returncode != 0 or not out_path.exists() or out_path.stat().st_size==0:
            raise RuntimeError("ffmpeg 生成透明 WebM 失敗")

        meta = {"hash":hid,"created":datetime.datetime.now().isoformat(timespec="seconds"),
                "w":gen_w,"h":gen_h,"fps":fps,"frames":frames,"kind":kind}
        meta_path.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")
        return out_path, (gen_w, gen_h)
    finally:
        try:
            for p in tmp_dir.glob("*.png"): p.unlink(missing_ok=True)
            tmp_dir.rmdir()
        except: pass

# ---------------- 後製（濾鏡 + Overlay） ----------------
def build_filter_chain_and_inputs(base_w, base_h, settings: dict, base_fps=25):
    post = (settings.get("post") or {})
    prof = settings.get("profile") or {}
    filters_base = []
    if (post.get("color") or {}).get("enabled", False):
        c = post["color"]; filters_base.append(f"eq=brightness={float(c.get('brightness',0.0))}:contrast={float(c.get('contrast',1.0))}:saturation={float(c.get('saturation',1.0))}")
    if (post.get("denoise") or {}).get("enabled", False):
        p = (post["denoise"].get("preset","medium") or "medium").lower()
        filters_base.append("hqdn3d=2:1.5:3:2" if p=="low" else ("hqdn3d=8:6:12:8" if p=="high" else "hqdn3d=4:3:6:4"))
    if (post.get("sharpen") or {}).get("enabled", False):
        amt = max(0.0, min(2.0, float(post["sharpen"].get("amount",0.8)))); filters_base.append(f"unsharp=5:5:{amt}:5:5:{amt/2:.3f}")
    if (post.get("fps_interp") or {}).get("enabled", False):
        tfps = int(post["fps_interp"].get("fps",60)); filters_base.append(f"minterpolate=fps={tfps}")

    graph_parts = []
    vcur = "v0"
    graph_parts.append(f"[0:v]{','.join(filters_base) if filters_base else 'null'}[{vcur}]")

    extra_inputs = []
    overlay_idx_input = 1
    overlays = post.get("overlays") or []
    simple_idx = 0

    for i,ov in enumerate(overlays, start=1):
        if not ov.get("enabled", False): 
            continue
        mode = (ov.get("mode") or "file").lower()
        op  = float(ov.get("opacity",1.0))
        if mode == "builtin_simple":
            simple_idx += 1
            simp = ov.get("simple") or {}
            kind = (simp.get("kind") or "snow").lower()
            dens = int(simp.get("density",30))
            size = int(simp.get("size",48))
            spd  = int(simp.get("speed",120))
            col  = simp.get("color","white")
            gnode, outlbl = builtin_simple_node(kind, base_w, base_h, base_fps, dens, size, spd, col, simple_idx, opacity=op)
            graph_parts.append(gnode)
            graph_parts.append(f"[{vcur}][{outlbl}]overlay=0:0:shortest=1[v{i}]")
            vcur = f"v{i}"
        elif mode == "builtin_advanced":
            adv = ov.get("advanced") or {}
            clip, (gen_w, gen_h) = _ensure_adv_overlay_clip(i, adv, base_w, base_h, base_fps, opacity=op, profile=prof)
            extra_inputs += ["-stream_loop","-1","-i", str(clip)]
            pre = []
            if gen_w!=base_w or gen_h!=base_h:
                pre.append(f"scale={base_w}:{base_h}:flags=lanczos")
            pre_chain = ",".join(pre) if pre else "null"
            graph_parts.append(f"[{overlay_idx_input}:v]{pre_chain}[ov{i}]")
            graph_parts.append(f"[{vcur}][ov{i}]overlay=0:0:shortest=1[v{i}]")
            vcur = f"v{i}"; overlay_idx_input += 1
        else:
            f = (ov.get("file") or "").strip()
            if not f or not (main_dir/f).exists():
                continue
            extra_inputs += ["-stream_loop","-1","-i", str(main_dir/f)]
            typ = (ov.get("type") or "alpha").lower()
            scl = bool(ov.get("scale_to_output",True))
            pre = ["format=rgba"]
            if scl: pre.append(f"scale={base_w}:{base_h}")
            pre_chain = ",".join(pre)
            if typ=="alpha":
                graph_parts.append(f"[{overlay_idx_input}:v]{pre_chain},colorchannelmixer=aa={op:.3f}[ov{i}]")
                graph_parts.append(f"[{vcur}][ov{i}]overlay=0:0:shortest=1[v{i}]"); vcur=f"v{i}"
            elif typ=="black":
                mode_b = (ov.get("black") or {}).get("mode","screen").lower()
                blend_mode = "screen" if mode_b=="screen" else "addition"
                graph_parts.append(f"[{overlay_idx_input}:v]{pre_chain}[ov{i}]")
                graph_parts.append(f"[{vcur}][ov{i}]blend=all_mode={blend_mode}:all_opacity={op:.3f}[v{i}]"); vcur=f"v{i}"
            else:
                gs = ov.get("greenscreen") or {}
                color = gs.get("color","0x00FF00"); sim=float(gs.get("similarity",0.20)); bl=float(gs.get("blend",0.10))
                graph_parts.append(f"[{overlay_idx_input}:v]chromakey={color}:{sim}:{bl},{pre_chain}[ov{i}]")
                graph_parts.append(f"[{vcur}][ov{i}]overlay=0:0:shortest=1[v{i}]"); vcur=f"v{i}"
            overlay_idx_input += 1

    vout = vcur
    filter_complex = ";".join(graph_parts)
    return extra_inputs, filter_complex, vout

import subprocess
from pathlib import Path
import shutil

def convert_webm_overlay_to_pngs(webm_path, png_prefix, frame_rate=25):
    """
    將 webm 特效檔解出 png 幀圖，回傳 png 路徑 pattern字串與目錄
    """
    webm_path = Path(webm_path)
    png_dir = webm_path.parent / "_overlaytemp"
    png_dir.mkdir(exist_ok=True)
    png_pattern = str(png_dir / f"{png_prefix}_%06d.png")
    for f in png_dir.glob(f"{png_prefix}_*.png"):
        f.unlink()
    cmd = [
        "ffmpeg", "-y", "-i", str(webm_path),
        "-vf", "pad=ceil(iw/2)*2:ceil(ih/2)*2",
        png_pattern
    ]
    subprocess.run(cmd, check=True)
    return png_pattern, png_dir

def cleanup_overlay_pngs(png_dir, png_prefix):
    for f in png_dir.glob(f"{png_prefix}_*.png"):
        f.unlink()
    try:
        png_dir.rmdir()
    except Exception:
        pass

def postprocess_final_video(base_plain: Path, settings: dict, out_path: Path, base_w=1920, base_h=1080, base_fps=25):
    post = (settings.get("post") or {})
    nothing = True
    if (post.get("color") or {}).get("enabled", False): nothing=False
    if (post.get("denoise") or {}).get("enabled", False): nothing=False
    if (post.get("sharpen") or {}).get("enabled", False): nothing=False
    if (post.get("fps_interp") or {}).get("enabled", False): nothing=False
    for ov in (post.get("overlays") or []):
        if ov.get("enabled", False): nothing=False
    if nothing:
        shutil.copyfile(base_plain, out_path); return True, "copy"

    extra_inputs, fc, vout = build_filter_chain_and_inputs(base_w, base_h, settings, base_fps=base_fps)

    # --- 新增判斷：若唯一 overlay 為 .webm，則轉成 png 疊圖 ---
    # 只補單一 webm overlay（如有多層可再遞迴；現在補你特效全黑問題）
    webm_overlay = None
    for e in extra_inputs:
        if isinstance(e, str) and e.lower().endswith(".webm"):
            webm_overlay = Path(e)
            break

    if webm_overlay and "overlay" in fc:
        # (1) 先拆 webm 成 png
        png_pattern, png_dir = convert_webm_overlay_to_pngs(webm_overlay, "adv_overlay", frame_rate=base_fps)
        # (2) 用 png 幀序列疊圖
        cmd = [
            "ffmpeg", "-y", "-i", str(base_plain),
            "-framerate", str(base_fps), "-i", png_pattern,
            "-filter_complex", "[0:v][1:v]overlay=0:0:shortest=1[v1]",
            "-map", "[v1]", "-map", "0:a?", "-c:v", "libx264", "-pix_fmt", "yuv420p", "-c:a", "aac", "-movflags", "+faststart", str(out_path)
        ]
        res = subprocess.run(cmd, capture_output=True, text=True)
        cleanup_overlay_pngs(png_dir, "adv_overlay")
        ok = (res.returncode==0 and out_path.exists() and out_path.stat().st_size>0)
        return ok, (res.stderr or "")[-400:] if not ok else "ok"
    else:
        # 沒有 webm overlay, 走原本路徑
        cmd = ["ffmpeg","-y","-i",str(base_plain)] + extra_inputs + [
            "-filter_complex", fc, "-map", f"[{vout}]", "-map", "0:a?","-c:v","libx264","-pix_fmt","yuv420p","-c:a","aac","-movflags","+faststart", str(out_path)
        ]
        res=_run(cmd, capture=True)
        ok = (res.returncode==0 and out_path.exists() and out_path.stat().st_size>0)
        return ok, (res.stderr or "")[-400:] if not ok else "ok"

# ---------------- 字幕硬燒/軟字幕 ----------------
def _double_escape_drive_colon(posix: str) -> str:
    if os.name != "nt": return posix
    if re.match(r'^[A-Za-z]:/', posix):
        return posix[0] + r'\\:/' + posix[3:]
    return posix
def _double_escape_path(p: Path) -> str:
    return _double_escape_drive_colon(p.as_posix())
def has_subtitles_filter():
    r=_run(["ffmpeg","-filters"], capture=True); blob=(r.stdout or "")+(r.stderr or ""); return "subtitles" in blob
def try_hardburn_subtitles(final_video: Path, ass_file: Path, fonts_root: Path, out_mp4: Path):
    if not has_subtitles_filter(): return False,"ffmpeg 缺少 subtitles 濾鏡(libass)"
    candidates=[fonts_root/"fonts", fonts_root]
    if os.name=="nt":
        sys_fonts=Path("C:/Windows/Fonts")
        if sys_fonts.exists(): candidates.append(sys_fonts)
    seen=set(); fonts_dirs=[]
    for c in candidates:
        key=c.resolve().as_posix().lower()
        if key not in seen:
            seen.add(key); fonts_dirs.append(c)
    ass_posix=ass_file.as_posix(); ass_double=_double_escape_path(ass_file)
    def variants(fd: Path):
        fd_posix=fd.as_posix(); fd_double=_double_escape_path(fd)
        return [
            ("double_escaped",       f"subtitles={ass_double}:fontsdir={fd_double}"),
            ("double_escaped+force", f"subtitles={ass_double}:fontsdir={fd_double}:force_style='FontName={FONTNAME_FORCE_STYLE}'"),
            ("plain",                f"subtitles={ass_posix}:fontsdir={fd_posix}"),
            ("plain+force",          f"subtitles={ass_posix}:fontsdir={fd_posix}:force_style='FontName={FONTNAME_FORCE_STYLE}'"),
            ("quoted",               f"subtitles='{ass_posix}':fontsdir='{fd_posix}'"),
        ]
    last_tail=""
    for fd in fonts_dirs:
        for label,vf in variants(fd):
            proc=_run(["ffmpeg","-y","-i",str(final_video),"-vf",vf,"-c:a","copy", str(out_mp4)], capture=True)
            if proc.returncode==0 and out_mp4.exists() and out_mp4.stat().st_size>0:
                return True,label
            last_tail=(proc.stderr or "")[-400:]
    return False,last_tail
def fallback_soft_sub(final_video: Path, ass_file: Path, out_mkv: Path):
    proc=_run(["ffmpeg","-y","-i",str(final_video),"-i",str(ass_file),
               "-map","0","-map","1","-c","copy", str(out_mkv)], capture=True)
    if proc.returncode==0 and out_mkv.exists() and out_mkv.stat().st_size>0:
        return True,"soft-sub ok"
    return False,(proc.stderr or "")[-400:]

# ---------------- 輸出流程 ----------------
def quickcheck(plan):
    lines=[]; scs=plan.get("scenes",[]); lines.append(f"[快速檢查] 場景數={len(scs)}")
    for sc in scs:
        num=sc.get("number"); af=sc.get("audio_file"); im=sc.get("image_file")
        aok=af and (main_dir/af).exists(); iok=im and (main_dir/im).exists()
        lines.append(f"場景{num:03d} 音訊={'OK' if aok else '缺'} 圖片={'OK' if iok else '缺'}")
    return "\n".join(lines)

def apply_profile_to_plan(plan: dict, settings: dict):
    prof = settings.get("profile") or {}
    outp = prof.get("output") or {}
    w = int(outp.get("w", 1920)); h=int(outp.get("h",1080)); fps=int(outp.get("fps",25))
    plan["global"]["resolution"]=[w,h]; plan["global"]["fps"]=fps
    return w,h,fps

def render_from_plan(plan, log_output: W.Output, settings: dict):
    with log_output: print("[輸出] 開始")
    # 用檔位覆蓋解析度/幀率
    w,h,fps = apply_profile_to_plan(plan, settings)
    events=build_subtitle_events(plan)
    ass_path=write_ass(events, plan, main_dir/"combined.ass", settings=settings)
    srt_path=write_srt(events, main_dir/"combined.srt")
    with log_output: print("[輸出] 字幕檔產生完成")

    # 逐場景 - 嚴格確保每個場景的圖片/音訊時長匹配
    clips=[]; total=0.0
    total_scenes = len(plan.get("scenes",[]))
    for idx, sc in enumerate(plan.get("scenes",[]), 1):
        num=sc.get("number")
        with log_output: print(f"[處理] 場景 {num:03d}/{total_scenes:03d} - 開始處理...")
        final_audio=process_scene_audio_return_final(sc, fallback_sec=DEFAULT_FALLBACK_DURATION)
        dur=_ffprobe_duration(final_audio) or sc.get("duration_override") or DEFAULT_FALLBACK_DURATION
        total+=dur
        clip=main_dir/f"scene_{num:03d}_clip.mp4"
        with log_output: print(f"[處理] 場景 {num:03d} - 音訊長度: {dur:.2f}秒, 生成片段中...")
        ok,label=make_scene_clip(sc.get("image_file"), final_audio.name, dur, w, h, fps, clip)
        if not ok: 
            with log_output: print(f"[錯誤] 場景 {num:03d} - 片段生成失敗!")
            raise RuntimeError(f"場景 {num} 片段失敗")
        with log_output: print(f"[完成] 場景 {num:03d} - 成功生成 ({label}) ✓")
        clips.append(clip)

    # 串接
    concat=main_dir/"concat.txt"
    with concat.open("w",encoding="utf-8") as f:
        for c in clips: f.write(f"file '{c.as_posix()}'\n")
    base=main_dir/"base_plain.mp4"
    res=_run(["ffmpeg","-y","-f","concat","-safe","0","-i",str(concat),"-c","copy", str(base)], capture=True)
    if not base.exists() or base.stat().st_size==0:
        with log_output: print("[輸出] 串接copy失敗 → 改重編碼")
        _run(["ffmpeg","-y","-f","concat","-safe","0","-i",str(concat),
              "-c:v","libx264","-pix_fmt","yuv420p","-c:a","aac", str(base)], capture=True)
        if not base.exists() or base.stat().st_size==0:
            raise RuntimeError("串接失敗兩次")

    # 後製（含內建簡易/進階/檔案 Overlay）
    post_out = main_dir/"output_final.mp4"
    ok_pp, msg = postprocess_final_video(base, settings, post_out, base_w=w, base_h=h, base_fps=fps)
    if not ok_pp:
        with log_output: print("[後製] 失敗，訊息尾段：", msg); shutil.copyfile(base, post_out)
    else:
        with log_output: print("[後製] 已套用：", msg)

    # 字幕硬燒 / 軟字幕
    hard_sub=main_dir/"output_final_subtitled.mp4"
    ok,msg=try_hardburn_subtitles(post_out, ass_path, main_dir, hard_sub)
    if not ok:
        with log_output: print("[字幕] 硬燒失敗 → 改封裝軟字幕", msg)
        soft_mkv=main_dir/"output_final_subtitled.mkv"
        ok2,msg2=fallback_soft_sub(post_out, ass_path, soft_mkv)
        with log_output: print("[字幕] 軟字幕：", "OK" if ok2 else msg2)
    else:
        with log_output: print("[字幕] 硬燒 OK：", msg)

    # 輕量預覽
    preview=main_dir/"output_final_preview.mp4"
    _run(["ffmpeg","-y","-i",str(post_out),
          "-c:v","libx264","-crf","30","-preset","veryfast",
          "-c:a","aac","-b:a","128k", str(preview)])
    with log_output: print(f"[完成] 總長度：約 {total:.2f} 秒")

# ---------------- 步驟0：全域設定（含一鍵檔位） ----------------
state = {
    "plan": None,
    "images_list": gather_images(),
    "audios_list": gather_audios(),
    "overlay_list": gather_overlay_candidates(),
    "settings": load_settings()
}

# TTS 區塊
tts_provider = W.Dropdown(options=[("OpenAI","OPENAI"),("Azure","AZURE"),("MiniMax","MINIMAX")],
                          description="TTS 服務：", value=state["settings"]["tts"]["provider"])
openai_model = W.Text(value=state["settings"]["tts"]["openai"]["model"], description="模型(型號)：", layout=W.Layout(width="240px"))
openai_voice = W.Text(value=state["settings"]["tts"]["openai"]["voice"], description="聲音(音色)：", layout=W.Layout(width="240px"))
openai_fmt   = W.Dropdown(options=[("MP3","mp3"),("WAV","wav")], value=state["settings"]["tts"]["openai"]["format"], description="格式：")
azure_region= W.Text(value=state["settings"]["tts"]["azure"].get("region",""), description="區域(region)：", layout=W.Layout(width="220px"))
azure_voice = W.Text(value=state["settings"]["tts"]["azure"]["voice"], description="聲音(音色)：", layout=W.Layout(width="240px"))
azure_rate  = W.IntSlider(value=int(state["settings"]["tts"]["azure"]["rate_pct"]), min=-50, max=50, step=1, description="語速(%)：", continuous_update=False)
azure_pitch = W.IntSlider(value=int(state["settings"]["tts"]["azure"]["pitch_pct"]), min=-50, max=50, step=1, description="音高(%)：", continuous_update=False)
azure_fmt   = W.Dropdown(options=[("24kHz/48k單聲道MP3","audio-24khz-48kbitrate-mono-mp3"),
                                  ("48kHz/192k單聲道MP3","audio-48khz-192kbitrate-mono-mp3"),
                                  ("24kHz/16bit單聲道PCM","riff-24khz-16bit-mono-pcm")],
                         value=state["settings"]["tts"]["azure"]["format"], description="格式：")
minimax_model = W.Text(value=state["settings"]["tts"]["minimax"]["model"], description="模型(型號)：", layout=W.Layout(width="240px"))
minimax_voice = W.Text(value=state["settings"]["tts"]["minimax"]["voice_id"], description="聲音ID：", layout=W.Layout(width="240px"))
minimax_fmt   = W.Dropdown(options=[("MP3","mp3"),("WAV","wav")], value=state["settings"]["tts"]["minimax"]["format"], description="格式：")

# 字幕
subs_scale = W.IntSlider(value=int(state["settings"]["subs"]["scale_pct"]), min=50, max=200, step=5, description="字幕大小(%)：", continuous_update=False)

# 後製基本
denoise_enabled = W.Checkbox(value=state["settings"]["post"]["denoise"]["enabled"], description="去雜訊(hqdn3d)")
denoise_preset  = W.Dropdown(options=[("低","low"),("中","medium"),("高","high")], value=state["settings"]["post"]["denoise"]["preset"], description="強度：")
sharpen_enabled = W.Checkbox(value=state["settings"]["post"]["sharpen"]["enabled"], description="銳利化(unsharp)")
sharpen_amount  = W.FloatSlider(value=float(state["settings"]["post"]["sharpen"]["amount"]), min=0.0, max=2.0, step=0.05, description="強度：", readout_format=".2f", continuous_update=False)
color_enabled   = W.Checkbox(value=state["settings"]["post"]["color"]["enabled"], description="顏色校正(eq)")
color_brightness= W.FloatSlider(value=float(state["settings"]["post"]["color"]["brightness"]), min=-0.5, max=0.5, step=0.01, description="亮度：", continuous_update=False)
color_contrast  = W.FloatSlider(value=float(state["settings"]["post"]["color"]["contrast"]), min=0.5, max=2.0, step=0.05, description="對比：", continuous_update=False)
color_saturation= W.FloatSlider(value=float(state["settings"]["post"]["color"]["saturation"]), min=0.0, max=2.0, step=0.05, description="飽和：", continuous_update=False)
fps_enabled     = W.Checkbox(value=state["settings"]["post"]["fps_interp"]["enabled"], description="補幀(變更幀率) minterpolate")
fps_target      = W.IntSlider(value=int(state["settings"]["post"]["fps_interp"]["fps"]), min=24, max=120, step=1, description="目標幀率(FPS)：", continuous_update=False)

# 覆蓋層（兩層）
def overlay_row(idx: int, ov: dict):
    enabled = W.Checkbox(value=ov.get("enabled", False), description=f"覆蓋層 {idx}：啟用")

    # 矯正 mode 值（修正舊設定 "builtin"；以及任何不在 options 的值）
    valid_modes = ("file", "builtin_simple", "builtin_advanced")
    mode_val = str(ov.get("mode", "builtin_simple") or "builtin_simple")
    if mode_val == "builtin":
        mode_val = "builtin_simple"
    if mode_val not in valid_modes:
        mode_val = "builtin_simple"
    mode_dd = W.Dropdown(
        options=[("檔案","file"),("內建(簡易)","builtin_simple"),("內建(進階)","builtin_advanced")],
        value=mode_val,
        description="模式："
    )

    opacity = W.FloatSlider(value=float(ov.get("opacity",1.0)), min=0.0, max=1.0, step=0.05, description="透明度：", continuous_update=False)
    scale   = W.Checkbox(value=ov.get("scale_to_output",True), description="跟隨輸出大小")

    # 檔案下拉：保證預設值存在於 options
    files_options = state["overlay_list"]
    file_val = ov.get("file","") or ""
    if file_val not in files_options:
        file_val = ""
    file_dd = W.Dropdown(options=files_options, value=file_val, description="檔案：")

    # 檔案類型：alpha/black/green
    valid_types = ("alpha","black","green")
    typ_val = ov.get("type","alpha")
    if typ_val not in valid_types:
        typ_val = "alpha"
    typ_dd  = W.Dropdown(options=[("透明(含Alpha)","alpha"),("黑底","black"),("綠幕","green")], value=typ_val, description="檔案類型：")
    # 黑底混合
    valid_black_modes = ("screen","addition")
    black_mode_val = (ov.get("black") or {}).get("mode","screen")
    if black_mode_val not in valid_black_modes:
        black_mode_val = "screen"
    black_mode = W.Dropdown(options=[("變亮(Screen)","screen"),("相加(Addition)","addition")], value=black_mode_val, description="黑底混合：")

    # 綠幕去背
    green_cfg = ov.get("greenscreen") or {}
    green_color= W.Text(value=green_cfg.get("color","0x00FF00"), description="去背顏色：")
    green_sim  = W.FloatSlider(value=float(green_cfg.get("similarity",0.20)), min=0.0, max=1.0, step=0.01, description="相似度：", continuous_update=False)
    green_blend= W.FloatSlider(value=float(green_cfg.get("blend",0.10)), min=0.0, max=1.0, step=0.01, description="融合：", continuous_update=False)
    file_box = W.VBox([W.HBox([file_dd, typ_dd, black_mode]), W.HBox([green_color, green_sim, green_blend])])

    # 內建(簡易)
    simp = ov.get("simple") or {}
    valid_kinds = ("snow","rain","leaves")
    simple_kind_val = simp.get("kind","snow")
    if simple_kind_val not in valid_kinds:
        simple_kind_val = "snow"
    simple_kind = W.Dropdown(options=[("雪","snow"),("雨","rain"),("樹葉","leaves")], value=simple_kind_val, description="類型：")
    simple_dens = W.IntSlider(value=int(simp.get("density",30)), min=1, max=200, step=1, description="密度：", continuous_update=False)
    simple_size = W.IntSlider(value=int(simp.get("size",48)), min=8, max=200, step=1, description="大小(px)：", continuous_update=False)
    simple_speed= W.IntSlider(value=int(simp.get("speed",120)), min=10, max=600, step=5, description="速度：", continuous_update=False)
    simple_color= W.Text(value=simp.get("color","white"), description="顏色：")
    simple_box = W.HBox([simple_kind, simple_dens, simple_size, simple_speed, simple_color])

    # 內建(進階)
    adv = ov.get("advanced") or {}
    adv_kind_val = adv.get("kind","snow")
    if adv_kind_val not in valid_kinds:
        adv_kind_val = "snow"
    adv_kind = W.Dropdown(options=[("雪","snow"),("雨","rain"),("樹葉","leaves")], value=adv_kind_val, description="類型：")
    adv_density = W.IntSlider(value=int(adv.get("density",60)), min=1, max=200, step=1, description="數量：", continuous_update=False)
    adv_size_min= W.IntSlider(value=int(adv.get("size_min",20)), min=6, max=200, step=1, description="最小大小：", continuous_update=False)
    adv_size_max= W.IntSlider(value=int(adv.get("size_max",64)), min=8, max=240, step=1, description="最大大小：", continuous_update=False)
    adv_spd_min = W.IntSlider(value=int(adv.get("speed_min",60)), min=10, max=600, step=5, description="最慢速度：", continuous_update=False)
    adv_spd_max = W.IntSlider(value=int(adv.get("speed_max",180)), min=10, max=800, step=5, description="最快速度：", continuous_update=False)
    adv_drift   = W.IntSlider(value=int(adv.get("drift",60)), min=0, max=200, step=1, description="左右飄幅：", continuous_update=False)
    adv_turb    = W.IntSlider(value=int(adv.get("turbulence",25)), min=0, max=200, step=1, description="亂流：", continuous_update=False)
    adv_rot_min = W.IntSlider(value=int(adv.get("rotation_min",-90)), min=-360, max=360, step=5, description="最小旋轉：", continuous_update=False)
    adv_rot_max = W.IntSlider(value=int(adv.get("rotation_max", 90)), min=-360, max=360, step=5, description="最大旋轉：", continuous_update=False)
    adv_loop    = W.IntSlider(value=int(adv.get("loop_sec",8)), min=2, max=20, step=1, description="循環秒數：", continuous_update=False)
    adv_seed    = W.IntText(value=int(adv.get("seed",1337)), description="隨機種子：")

    # 自訂貼圖下拉：確保值存在於 options
    adv_tex_options = [""] + [p.name for p in main_dir.glob("*.png")]
    adv_tex_val = Path(adv.get("texture_file","")).name if adv.get("texture_file") else ""
    if adv_tex_val not in adv_tex_options:
        adv_tex_val = ""
    adv_tex     = W.Dropdown(options=adv_tex_options, value=adv_tex_val, description="自訂貼圖：")

    adv_box1    = W.HBox([adv_kind, adv_density, adv_size_min, adv_size_max, adv_spd_min, adv_spd_max])
    adv_box2    = W.HBox([adv_drift, adv_turb, adv_rot_min, adv_rot_max, adv_loop, adv_seed, adv_tex])

    builtin_simple_box  = W.VBox([simple_box])
    builtin_advanced_box= W.VBox([adv_box1, adv_box2])

    def on_mode_change(change):
        m = change["new"]
        file_box.layout.display    = None if m=="file" else "none"
        builtin_simple_box.layout.display   = None if m=="builtin_simple" else "none"
        builtin_advanced_box.layout.display = None if m=="builtin_advanced" else "none"
    mode_dd.observe(on_mode_change, names="value")
    on_mode_change({"new": mode_dd.value})

    row = W.VBox([
        W.HBox([enabled, mode_dd, opacity, scale]),
        file_box,
        builtin_simple_box,
        builtin_advanced_box
    ], layout=W.Layout(border="1px solid #ddd", padding="6px"))
    return {
        "widget": row,
        "enabled": enabled,
        "mode_dd": mode_dd,
        "opacity": opacity,
        "scale": scale,
        # file
        "file_dd": file_dd, "typ_dd": typ_dd, "black_mode": black_mode,
        "green_color": green_color, "green_sim": green_sim, "green_blend": green_blend,
        # simple
        "simple_kind": simple_kind, "simple_dens": simple_dens, "simple_size": simple_size, "simple_speed": simple_speed, "simple_color": simple_color,
        # advanced
        "adv_kind": adv_kind, "adv_density": adv_density, "adv_size_min": adv_size_min, "adv_size_max": adv_size_max,
        "adv_spd_min": adv_spd_min, "adv_spd_max": adv_spd_max, "adv_drift": adv_drift, "adv_turb": adv_turb,
        "adv_rot_min": adv_rot_min, "adv_rot_max": adv_rot_max, "adv_loop": adv_loop, "adv_seed": adv_seed, "adv_tex": adv_tex
    }

ov_widgets = []
post_ovs = state["settings"]["post"]["overlays"]
for i in range(2):
    ov_widgets.append(overlay_row(i+1, post_ovs[i] if i < len(post_ovs) else {}))

# 設定按鈕/狀態
settings_status = W.Output(layout=W.Layout(border="1px solid #ccc", max_height="160px", overflow="auto"))
btn_settings_save = W.Button(description="儲存設定", button_style="success", icon="save")
btn_settings_reload = W.Button(description="重新讀取", icon="refresh")
btn_settings_reset = W.Button(description="回預設", icon="undo")

# 一鍵檔位：省資源/均衡/頂規
btn_prof_eco = W.Button(description="一鍵：省資源", icon="leaf", button_style="")
btn_prof_bal = W.Button(description="一鍵：均衡", icon="adjust", button_style="info")
btn_prof_ultra=W.Button(description="一鍵：頂規", icon="star", button_style="warning")
lbl_prof = W.Label(value=f"目前檔位：{(state['settings'].get('profile') or {}).get('name','eco')}")

def apply_profile_to_ui(name: str):
    # 讀現有設定做基底
    s = load_settings()
    s["profile"]["name"] = name
    if name=="eco":
        s["profile"]["output"] = {"w":1280,"h":720,"fps":25}
        s["profile"]["adv_half_res"] = True
        s["profile"]["vp9"] = {"crf":32,"cpu_used":8,"tile_columns":2,"row_mt":1,"auto_alt_ref":0}
        s["post"]["denoise"]["enabled"]=False
        s["post"]["sharpen"]["enabled"]=False
        s["post"]["color"]["enabled"]=False
        s["post"]["fps_interp"]["enabled"]=False
        if s["post"]["overlays"]:
            s["post"]["overlays"][0].update({
                "enabled": True, "mode":"builtin_simple", "opacity":1.0,
                "simple":{"kind":"snow","density":35,"size":36,"speed":130,"color":"white"},
            })
        if len(s["post"]["overlays"])>1:
            s["post"]["overlays"][1]["enabled"]=False
    elif name=="balanced":
        s["profile"]["output"] = {"w":1920,"h":1080,"fps":25}
        s["profile"]["adv_half_res"] = True
        s["profile"]["vp9"] = {"crf":30,"cpu_used":6,"tile_columns":2,"row_mt":1,"auto_alt_ref":0}
        s["post"]["denoise"]["enabled"]=False
        s["post"]["sharpen"]["enabled"]=False
        s["post"]["color"]["enabled"]=False
        s["post"]["fps_interp"]["enabled"]=False
        if s["post"]["overlays"]:
            s["post"]["overlays"][0].update({
                "enabled": True, "mode":"builtin_advanced", "opacity":1.0,
                "advanced":{"kind":"snow","density":50,"size_min":24,"size_max":64,
                            "speed_min":60,"speed_max":160,"drift":60,"turbulence":25,
                            "rotation_min":-60,"rotation_max":60,"layers":{"near":0.4,"mid":0.35,"far":0.25},
                            "loop_sec":8,"seed":1337,"texture_file":""}
            })
        if len(s["post"]["overlays"])>1:
            s["post"]["overlays"][1].update({"enabled": False})
    else:  # ultra
        s["profile"]["output"] = {"w":1920,"h":1080,"fps":25}  # 想要 4K 可改 "w":3840,"h":2160
        s["profile"]["adv_half_res"] = False
        s["profile"]["vp9"] = {"crf":28,"cpu_used":4,"tile_columns":2,"row_mt":1,"auto_alt_ref":0}
        s["post"]["denoise"]["enabled"]=False
        s["post"]["sharpen"]["enabled"]=False
        s["post"]["color"]["enabled"]=False
        s["post"]["fps_interp"]["enabled"]=False
        if s["post"]["overlays"]:
            s["post"]["overlays"][0].update({
                "enabled": True, "mode":"builtin_advanced", "opacity":1.0,
                "advanced":{"kind":"snow","density":90,"size_min":24,"size_max":80,
                            "speed_min":60,"speed_max":200,"drift":90,"turbulence":35,
                            "rotation_min":-120,"rotation_max":120,"layers":{"near":0.4,"mid":0.35,"far":0.25},
                            "loop_sec":12,"seed":2024,"texture_file":""}
            })
        if len(s["post"]["overlays"])>1:
            s["post"]["overlays"][1].update({
                "enabled": True, "mode":"builtin_advanced", "opacity":0.9,
                "advanced":{"kind":"leaves","density":36,"size_min":24,"size_max":72,
                            "speed_min":80,"speed_max":160,"drift":90,"turbulence":35,
                            "rotation_min":-120,"rotation_max":120,"layers":{"near":0.5,"mid":0.3,"far":0.2},
                            "loop_sec":8,"seed":2025,"texture_file":""}
            })
    # 關鍵：先存檔，再 reload 面板，避免被舊檔案覆蓋
    save_settings(s)
    state["settings"]=s
    on_settings_reload()  # 這次會讀到剛存的值
    lbl_prof.value=f"目前檔位：{name}"
    with settings_status:
        clear_output()
        print(f"[檔位] 已套用「{name}」並寫入 settings.json（面板已同步）。")
        
btn_prof_eco.on_click(lambda _: apply_profile_to_ui("eco"))
btn_prof_bal.on_click(lambda _: apply_profile_to_ui("balanced"))
btn_prof_ultra.on_click(lambda _: apply_profile_to_ui("ultra"))

def collect_settings_from_ui():
    s = load_settings()
    # TTS
    s["tts"]["provider"] = tts_provider.value
    s["tts"]["openai"]["model"] = openai_model.value.strip() or "tts-1"
    s["tts"]["openai"]["voice"] = openai_voice.value.strip() or "alloy"
    s["tts"]["openai"]["format"]= openai_fmt.value
    s["tts"]["azure"]["region"] = azure_region.value.strip()
    s["tts"]["azure"]["voice"]  = azure_voice.value.strip() or "zh-TW-HsiaoChenNeural"
    s["tts"]["azure"]["rate_pct"]= int(azure_rate.value)
    s["tts"]["azure"]["pitch_pct"]= int(azure_pitch.value)
    s["tts"]["azure"]["format"] = azure_fmt.value
    s["tts"]["minimax"]["model"]= minimax_model.value.strip() or "speech-01"
    s["tts"]["minimax"]["voice_id"]= minimax_voice.value.strip() or "female-1"
    s["tts"]["minimax"]["format"]= minimax_fmt.value
    # Subs
    s["subs"]["scale_pct"]= int(subs_scale.value)
    # Post basics
    s["post"]["denoise"]["enabled"] = bool(denoise_enabled.value)
    s["post"]["denoise"]["preset"]  = denoise_preset.value
    s["post"]["sharpen"]["enabled"] = bool(sharpen_enabled.value)
    s["post"]["sharpen"]["amount"]  = float(sharpen_amount.value)
    s["post"]["color"]["enabled"]   = bool(color_enabled.value)
    s["post"]["color"]["brightness"]= float(color_brightness.value)
    s["post"]["color"]["contrast"]  = float(color_contrast.value)
    s["post"]["color"]["saturation"]= float(color_saturation.value)
    s["post"]["fps_interp"]["enabled"]= bool(fps_enabled.value)
    s["post"]["fps_interp"]["fps"]    = int(fps_target.value)
    # Overlays
    ovs=[]
    for ow in ov_widgets:
        adv_tex_name = ow["adv_tex"].value or ""
        adv_tex_path = (main_dir/adv_tex_name).as_posix() if adv_tex_name else ""
        ov={
            "enabled": bool(ow["enabled"].value),
            "mode": ow["mode_dd"].value,
            "file": ow["file_dd"].value or "",
            "type": ow["typ_dd"].value,
            "opacity": float(ow["opacity"].value),
            "scale_to_output": bool(ow["scale"].value),
            "black": {"mode": ow["black_mode"].value},
            "greenscreen": {
                "color": ow["green_color"].value or "0x00FF00",
                "similarity": float(ow["green_sim"].value),
                "blend": float(ow["green_blend"].value)
            },
            "simple": {
                "kind": ow["simple_kind"].value,
                "density": int(ow["simple_dens"].value),
                "size": int(ow["simple_size"].value),
                "speed": int(ow["simple_speed"].value),
                "color": ow["simple_color"].value or "white"
            },
            "advanced": {
                "kind": ow["adv_kind"].value,
                "density": int(ow["adv_density"].value),
                "size_min": int(ow["adv_size_min"].value),
                "size_max": int(ow["adv_size_max"].value),
                "speed_min": int(ow["adv_spd_min"].value),
                "speed_max": int(ow["adv_spd_max"].value),
                "drift": float(ow["adv_drift"].value),
                "turbulence": float(ow["adv_turb"].value),
                "rotation_min": float(ow["adv_rot_min"].value),
                "rotation_max": float(ow["adv_rot_max"].value),
                "layers": {"near":0.4,"mid":0.35,"far":0.25},  # 固定比例（需要UI再開）
                "loop_sec": int(ow["adv_loop"].value),
                "seed": int(ow["adv_seed"].value),
                "texture_file": adv_tex_path
            }
        }
        ovs.append(ov)
    s["post"]["overlays"]= ovs
    # 檔位名維持
    if "profile" not in s: s["profile"] = default_settings()["profile"]
    return s

def on_settings_save(_b=None):
    s = collect_settings_from_ui()
    save_settings(s); state["settings"] = s
    with settings_status:
        clear_output()
        print("[設定] 已儲存到 settings.json")
        print("目前檔位：", (s.get("profile") or {}).get("name","eco"))
        print("ffmpeg 可用:", bool(shutil.which("ffmpeg")))

def on_settings_reload(_b=None):
    s = load_settings(); state["settings"] = s
    tts_provider.value = s["tts"]["provider"]
    openai_model.value = s["tts"]["openai"]["model"]; openai_voice.value = s["tts"]["openai"]["voice"]; openai_fmt.value=s["tts"]["openai"]["format"]
    azure_region.value = s["tts"]["azure"].get("region",""); azure_voice.value=s["tts"]["azure"]["voice"]; azure_rate.value=int(s["tts"]["azure"]["rate_pct"]); azure_pitch.value=int(s["tts"]["azure"]["pitch_pct"]); azure_fmt.value=s["tts"]["azure"]["format"]
    minimax_model.value=s["tts"]["minimax"]["model"]; minimax_voice.value=s["tts"]["minimax"]["voice_id"]; minimax_fmt.value=s["tts"]["minimax"]["format"]
    subs_scale.value=int(s["subs"]["scale_pct"])
    denoise_enabled.value=s["post"]["denoise"]["enabled"]; denoise_preset.value=s["post"]["denoise"]["preset"]
    sharpen_enabled.value=s["post"]["sharpen"]["enabled"]; sharpen_amount.value=float(s["post"]["sharpen"]["amount"])
    color_enabled.value=s["post"]["color"]["enabled"]; color_brightness.value=float(s["post"]["color"]["brightness"]); color_contrast.value=float(s["post"]["color"]["contrast"]); color_saturation.value=float(s["post"]["color"]["saturation"])
    fps_enabled.value=s["post"]["fps_interp"]["enabled"]; fps_target.value=int(s["post"]["fps_interp"]["fps"])
    # overlays
    state["overlay_list"] = gather_overlay_candidates()
    for i,ow in enumerate(ov_widgets):
        ov = s["post"]["overlays"][i] if i < len(s["post"]["overlays"]) else {}
        ow["enabled"].value = ov.get("enabled",False)
        ow["mode_dd"].value = ov.get("mode","builtin_simple")
        ow["file_dd"].options = state["overlay_list"]; ow["file_dd"].value = ov.get("file","") if ov.get("file","") in ow["file_dd"].options else ""
        ow["typ_dd"].value  = ov.get("type","alpha")
        ow["black_mode"].value = (ov.get("black") or {}).get("mode","screen")
        ow["green_color"].value= (ov.get("greenscreen") or {}).get("color","0x00FF00")
        ow["green_sim"].value  = float((ov.get("greenscreen") or {}).get("similarity",0.20))
        ow["green_blend"].value= float((ov.get("greenscreen") or {}).get("blend",0.10))
        simp = ov.get("simple") or {}
        ow["simple_kind"].value = simp.get("kind","snow")
        ow["simple_dens"].value = int(simp.get("density",30))
        ow["simple_size"].value = int(simp.get("size",48))
        ow["simple_speed"].value= int(simp.get("speed",120))
        ow["simple_color"].value= simp.get("color","white")
        adv = ov.get("advanced") or {}
        ow["adv_kind"].value = adv.get("kind","snow")
        ow["adv_density"].value = int(adv.get("density",60))
        ow["adv_size_min"].value = int(adv.get("size_min",20))
        ow["adv_size_max"].value = int(adv.get("size_max",64))
        ow["adv_spd_min"].value  = int(adv.get("speed_min",60))
        ow["adv_spd_max"].value  = int(adv.get("speed_max",180))
        ow["adv_drift"].value    = int(adv.get("drift",60))
        ow["adv_turb"].value     = int(adv.get("turbulence",25))
        ow["adv_rot_min"].value  = int(adv.get("rotation_min",-90))
        ow["adv_rot_max"].value  = int(adv.get("rotation_max", 90))
        ow["adv_loop"].value     = int(adv.get("loop_sec",8))
        ow["adv_seed"].value     = int(adv.get("seed",1337))
        ow["adv_tex"].options = [""] + [p.name for p in main_dir.glob("*.png")]
        cur_tex = Path(adv.get("texture_file","")).name if adv.get("texture_file") else ""
        ow["adv_tex"].value = cur_tex if cur_tex in ow["adv_tex"].options else ""
    lbl_prof.value=f"目前檔位：{(s.get('profile') or {}).get('name','eco')}"
    with settings_status:
        clear_output(); print("[設定] 已重新讀取 settings.json")

def on_settings_reset(_b=None):
    s = default_settings(); save_settings(s); state["settings"]=s; on_settings_reload()

btn_settings_save.on_click(on_settings_save)
btn_settings_reload.on_click(on_settings_reload)
btn_settings_reset.on_click(on_settings_reset)

# UI 群組
openai_box = W.HBox([openai_model, openai_voice, openai_fmt])
azure_box  = W.HBox([azure_region, azure_voice, azure_fmt])
azure_box2 = W.HBox([azure_rate, azure_pitch])
minimax_box= W.HBox([minimax_model, minimax_voice, minimax_fmt])
subs_box = W.HBox([subs_scale])
post_row1 = W.HBox([denoise_enabled, denoise_preset, sharpen_enabled, sharpen_amount])
post_row2 = W.HBox([color_enabled, color_brightness, color_contrast, color_saturation])
post_row3 = W.HBox([fps_enabled, fps_target])
profile_row = W.HBox([btn_prof_eco, btn_prof_bal, btn_prof_ultra, lbl_prof])

settings_ui = W.VBox([
    W.HTML("<b>步驟0：全域設定（前製 TTS / 後製效果 / 字幕大小 / 一鍵檔位）</b>"),
    W.HTML("<u>一鍵檔位</u>（點選後面板會更新，記得按「儲存設定」）"),
    profile_row,
    W.HTML("<u>TTS 語音設定</u>"),
    tts_provider, W.HTML("OpenAI"), openai_box, W.HTML("Azure"), azure_box, azure_box2, W.HTML("MiniMax（骨架）"), minimax_box,
    W.HTML("<u>字幕大小（百分比，ASS Style ScaleX/ScaleY）</u>"), subs_box,
    W.HTML("<u>後製效果（ffmpeg 濾鏡）</u>"), post_row1, post_row2, post_row3,
    W.HTML("<i>飄落特效 / 覆蓋層（最多 2 層；模式可選 檔案 / 內建(簡易) / 內建(進階)）</i>"),
    ov_widgets[0]["widget"], ov_widgets[1]["widget"],
    W.HBox([btn_settings_save, btn_settings_reload, btn_settings_reset]),
    settings_status,
    W.HTML("<hr>")
])
display(settings_ui)

# ---------------- 步驟1：分鏡編輯器 ----------------
status_out=W.Output(layout=W.Layout(border="1px solid #ccc", max_height="200px", overflow="auto"))
log_out   =W.Output(layout=W.Layout(border="1px solid #ccc", max_height="260px", overflow="auto"))

ta_text   =W.Textarea(value="", placeholder="貼上腳本：每行=一個分鏡(Scene)", layout=W.Layout(width="100%", height="120px"))
btn_import=W.Button(description="載入文字(重建)", button_style="info", icon="upload")
btn_qc    =W.Button(description="快速檢查", icon="search")
btn_save  =W.Button(description="儲存規劃檔", button_style="success", icon="save")
btn_pre   =W.Button(description="檔案檢查", icon="check")
btn_render=W.Button(description="輸出影片", button_style="primary", icon="film")
btn_refresh_img=W.Button(description="重整圖片清單", icon="refresh")
btn_refresh_aud=W.Button(description="重整音訊清單", icon="refresh")

scenes_box=W.VBox([])

def rebuild_scenes_ui():
    rows=[]; plan=state["plan"]
    if not plan: scenes_box.children=[]; return
    imgs=gather_images(); auds=gather_audios()
    for sc in plan.get("scenes",[]):
        num=sc["number"]
        preview=W.Output(layout=W.Layout(border="1px solid #ccc", width="200px", height="160px"))
        with preview:
            imgf=sc.get("image_file")
            if imgf and (main_dir/imgf).exists():
                from IPython.display import Image as IPyImage; display(IPyImage(filename=str(main_dir/imgf)))
            else:
                print("(尚無圖片)")
        sub_area=W.Textarea(value=sc.get("subtitle_text") or sc.get("raw_text") or "", placeholder=f"字幕 (場景{num})", layout=W.Layout(width="100%", height="70px"))
        audio_dd=W.Dropdown(options=auds, value=sc.get("audio_file") or "", description=f"音訊 {num}：", layout=W.Layout(width="280px"))
        tts_btn=W.Button(description="產生語音(TTS)", icon="microphone", layout=W.Layout(width="130px"))
        audio_play=W.Output(layout=W.Layout(border="1px solid #eee", width="100%", height="70px"))
        def refresh_audio_player():
            with audio_play:
                clear_output()
                af=sc.get("audio_file")
                if af and (main_dir/af).exists():
                    from IPython.display import Audio as IPyAudio; display(IPyAudio(filename=str(main_dir/af), autoplay=False))
                else:
                    print("(尚無音訊)")
        prompt_area=W.Textarea(value=sc.get("ai_prompt") or "", placeholder=f"圖片提示(場景{num})", layout=W.Layout(width="100%", height="70px"))
        image_dd=W.Dropdown(options=imgs, value=sc.get("image_file") or "", description=f"圖片 {num}：", layout=W.Layout(width="280px"))
        img_btn=W.Button(description="AI 產生圖片", icon="image", layout=W.Layout(width="120px"))

        def on_sub_change(change, sc=sc): sc["subtitle_text"]=change["new"]; sc["raw_text"]=change["new"]
        sub_area.observe(on_sub_change, names="value")
        def on_prompt_change(change, sc=sc): sc["ai_prompt"]=change["new"]
        prompt_area.observe(on_prompt_change, names="value")
        def on_audio_dd(change, sc=sc): sc["audio_file"]=change["new"] or None; refresh_audio_player()
        audio_dd.observe(on_audio_dd, names="value")
        def on_image_dd(change, sc=sc, preview=preview):
            sc["image_file"]=change["new"] or None
            with preview:
                clear_output()
                imgf=sc.get("image_file")
                if imgf and (main_dir/imgf).exists():
                    from IPython.display import Image as IPyImage; display(IPyImage(filename=str(main_dir/imgf)))
                else:
                    print("(尚無圖片)")
        image_dd.observe(on_image_dd, names="value")

        def on_tts_click(_b, sc=sc, audio_dd=audio_dd):
            text=sc.get("subtitle_text") or sc.get("raw_text") or ""
            outstem=main_dir/f"scene_{sc.get('number'):03d}_tts"
            ok,msg=tts_generate(text, outstem, state["settings"], default_voice=(sc.get("tts") or {}).get("voice","alloy"))
            if not ok:
                wav=outstem.with_suffix(".wav")
                _run(["ffmpeg","-y","-f","lavfi","-i","anullsrc=r=48000:cl=stereo","-t","2.5","-c:a","pcm_s16le", str(wav)])
                mp3=outstem.with_suffix(".mp3")
                _run(["ffmpeg","-y","-i",str(wav),"-c:a","libmp3lame","-q:a","4", str(mp3)])
            prov = state["settings"]["tts"]["provider"]
            tgt=None
            if prov=="OPENAI":
                fmt = state["settings"]["tts"]["openai"].get("format","mp3")
                candidate = outstem.with_suffix(f".{fmt}")
                tgt = candidate if candidate.exists() else (outstem.with_suffix(".mp3") if outstem.with_suffix(".mp3").exists() else None)
            elif prov=="AZURE":
                tgt = outstem.with_suffix(".mp3") if outstem.with_suffix(".mp3").exists() else None
            else:
                fmt = state["settings"]["tts"]["minimax"].get("format","mp3")
                candidate = outstem.with_suffix(f".{fmt}")
                tgt = candidate if candidate.exists() else (outstem.with_suffix(".mp3") if outstem.with_suffix(".mp3").exists() else None)
            sc["audio_file"] = tgt.name if tgt else sc.get("audio_file")
            audio_dd.options=gather_audios(); audio_dd.value=sc.get("audio_file") or ""
            refresh_audio_player()
        tts_btn.on_click(on_tts_click)

        def openai_image(prompt: str, out_path: Path, model="dall-e-3", size="1792x1024"):
            if not OPENAI_API_KEY: return False,"缺少 OPENAI_API_KEY"
            import requests, base64
            try:
                url="https://api.openai.com/v1/images/generations"
                headers={"Authorization":f"Bearer {OPENAI_API_KEY}","Content-Type":"application/json"}
                if OPENAI_ORG_ID: headers["OpenAI-Organization"]=OPENAI_ORG_ID
                payload={"model":model,"prompt":prompt or "","size":size,"response_format":"b64_json"}
                r=requests.post(url,headers=headers,json=payload,timeout=180)
                if r.status_code!=200: return False,f"{r.status_code} {r.text[:160]}"
                b64=(r.json().get("data") or [{}])[0].get("b64_json")
                if not b64: return False,"No b64"
                out_path.write_bytes(base64.b64decode(b64)); return True,"OK"
            except Exception as e:
                return False,f"Exception {e}"
                
        def placeholder_img_local(out_path: Path):
            try:
                from PIL import Image, ImageDraw
                img = Image.new("RGB", (1280, 720), (45, 45, 45))
                d = ImageDraw.Draw(img)
                d.text((24, 24), "Placeholder", fill=(255, 255, 255))
                tiny = (
                    b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x01\x00\x00\x00\x01"
                    b"\x08\x02\x00\x00\x00\x90wS\xde\x00\x00\x00\x0cIDAT\x08\xd7c\xf8\x0f\x00\x01"
                    b"\x01\x01\x00\x18\xdd\x8d\xe1\x00\x00\x00\x00IEND\xaeB`\x82"
                )
                try:
                    out_path.write_bytes(tiny)
                    return True
                except Exception:
                    return False
            except Exception:
                return False
            

        def on_img_click(_b, sc=sc, image_dd=image_dd, preview=preview):
            prompt=sc.get("ai_prompt") or sc.get("subtitle_text") or ""
            outp=main_dir/f"scene_{sc.get('number'):03d}_img.png"
            ok,msg=openai_image(prompt, outp, model="dall-e-3", size="1792x1024") if OPENAI_API_KEY else (placeholder_img_local(outp), "OK")
            sc["image_file"]=outp.name if outp.exists() else sc.get("image_file")
            image_dd.options=gather_images(); image_dd.value=sc.get("image_file") or ""
            with preview:
                clear_output()
                imgf=sc.get("image_file")
                if imgf and (main_dir/imgf).exists():
                    from IPython.display import Image as IPyImage; display(IPyImage(filename=str(main_dir/imgf)))
                else:
                    print("(尚無圖片)")
        img_btn.on_click(on_img_click)

        upper_row=W.HBox([sub_area, W.VBox([W.HBox([audio_dd, tts_btn]), audio_play], layout=W.Layout(width="380px"))], layout=W.Layout(width="100%", align_items="flex-start"))
        lower_row=W.HBox([prompt_area, W.HBox([image_dd, img_btn], layout=W.Layout(width="380px"))], layout=W.Layout(width="100%", align_items="flex-start"))
        row_panel=W.HBox([W.VBox([upper_row, lower_row], layout=W.Layout(width="100%")), preview], layout=W.Layout(width="100%", align_items="flex-start", justify_content="space-between"))

        refresh_audio_player(); rows.append(row_panel)
    scenes_box.children=rows

def do_import_text(_b=None):
    lines=[l.strip() for l in ta_text.value.splitlines() if l.strip()]
    if not lines: lines=ensure_text_lines()
    if not lines: 
        with status_out: clear_output(); print("沒有文字可建立分鏡。請貼上腳本或建立 text.txt。"); return
    plan=build_plan_from_lines(lines); state["plan"]=plan; rebuild_scenes_ui()
    with status_out: clear_output(); print("[載入] 已重建分鏡。")

def do_qc(_b=None):
    with status_out:
        clear_output()
        if not state["plan"]: print("尚未載入規劃檔。"); return
        print(quickcheck(state["plan"]))

def do_refresh_img(_b=None):
    rebuild_scenes_ui()
    with status_out:
        clear_output()
        print("[重整] 圖片清單已更新。")

def do_refresh_aud(_b=None):
    rebuild_scenes_ui()
    with status_out:
        clear_output()
        print("[重整] 音訊清單已更新。")

def do_preflight(_b=None):
    plan = state["plan"]
    with status_out:
        clear_output()
        if not plan:
            print("尚未載入規劃。"); return
        meta = {
            "已有 plan_current.yaml": (main_dir/PLAN_CURRENT).exists(),
            "已有 handshake": (main_dir/HANDSHAKE_NAME).exists(),
            "已有 lock": (main_dir/LOCK_NAME).exists(),
            "場景數": plan_scene_count(plan)
        }
        print("[檢查] ", meta)
        # 併入「快速檢查」
        print("\n[逐場景檢查]")
        print(quickcheck(plan))

def do_save_plan(_b=None):
    plan = state["plan"]
    if not plan:
        with status_out:
            clear_output()
            print("尚未有規劃內容，無法儲存。")
        return

    # 只警告，不阻擋儲存（允許分段作業）
    missing_audio = [sc.get("number") for sc in plan.get("scenes", [])
                     if not sc.get("audio_file") or not (main_dir / sc.get("audio_file")).exists()]

    # 向前繼承圖片（第一張之前不補，之後沒指定的用最近一張）
    last_img = None
    filled = 0
    for sc in plan.get("scenes", []):
        img = sc.get("image_file")
        if img and (main_dir / img).exists():
            last_img = img
        elif last_img:
            sc["image_file"] = last_img
            filled += 1

    # 寫檔（含備份與握手/鎖）
    plan_text = yaml.safe_dump(plan, allow_unicode=True)
    bname = backup_plan_yaml(plan_text)
    (main_dir / PLAN_CURRENT).write_text(plan_text, encoding="utf-8")
    md5 = write_handshake_lock(plan_text, plan_scene_count(plan))

    # 立即刷新 UI 讓你看到繼承結果
    rebuild_scenes_ui()

    # 狀態輸出：成功訊息 + 警告（若有）
    with status_out:
        clear_output()
        print(f"[儲存] plan_current.yaml 已寫入，場景={plan_scene_count(plan)} md5={md5[:8]}")
        print(f"[備份] {bname}")
        if filled > 0:
            print(f"已自動補齊缺圖（向前繼承）{filled} 個場景，UI 已刷新。")
        else:
            print("圖片向前繼承：無需補齊。")
        if missing_audio:
            print(f"⚠️ 警告：以下場景目前缺少音訊（允許先儲存）：{missing_audio}")
            print("    提示：可稍後補音再存，或在輸出前用「檔案檢查」確認。")
        else:
            print("音訊檢查：OK")
            
def do_render(_b=None):
    plan = state["plan"]
    if not plan:
        with status_out:
            clear_output()
            print("尚未載入規劃，無法輸出。")
        return

    # 核心變更：使用面板即時值，而非讀 settings.json
    state["settings"] = collect_settings_from_ui()

    # 仍照常落盤 plan / 寫 handshake（不影響設定是否儲存）
    plan_text = yaml.safe_dump(plan, allow_unicode=True)
    (main_dir/PLAN_CURRENT).write_text(plan_text, encoding="utf-8")
    write_handshake_lock(plan_text, plan_scene_count(plan))

    with log_out:
        clear_output()
        print("[輸出] 開始處理（使用面板即時設定，未必已儲存到 settings.json）...")
    try:
        render_from_plan(plan, log_out, settings=state["settings"])
    except Exception as e:
        with log_out:
            print("輸出失敗：", e)

btn_import.on_click(do_import_text); btn_qc.on_click(do_qc); btn_save.on_click(do_save_plan)
btn_pre.on_click(do_preflight); btn_render.on_click(do_render)
btn_refresh_img.on_click(do_refresh_img); btn_refresh_aud.on_click(do_refresh_aud)

controls_row1=W.HBox([btn_import, btn_qc, btn_refresh_img, btn_refresh_aud], layout=W.Layout(flex_flow="row wrap"))
controls_row2=W.HBox([btn_save, btn_pre, btn_render], layout=W.Layout(flex_flow="row wrap"))

ui=W.VBox([
    W.HTML("<b>步驟1：分鏡編輯</b> — 每個場景：上框字幕 + 右側音訊選擇/TTS（下一行播放器）；下框圖片提示 + 右側圖片選擇/AI生圖；最右側圖片預覽"),
    ta_text, controls_row1, controls_row2, W.HTML("<hr>"), scenes_box, W.HTML("<hr>"),
    W.HTML("<b>狀態</b>"), status_out, W.HTML("<b>輸出日誌</b>"), log_out
])
display(ui)

# 啟動時自動載入既有 plan
try:
    pc=main_dir/PLAN_CURRENT
    if pc.exists():
        loaded=yaml.safe_load(pc.read_text(encoding="utf-8"))
        if plan_scene_count(loaded)>0:
            state["plan"]=loaded; rebuild_scenes_ui()
            with status_out: clear_output(); print(f"[啟動] 已載入 plan_current.yaml 場景={plan_scene_count(loaded)}"); print(quickcheck(loaded))
except Exception as e:
    with status_out: clear_output(); print("載入既有規劃失敗：", e)

[初始化] 主目錄 = C:\Users\flyre\mynotebooks


Exception in thread Thread-31 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\flyre\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\flyre\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\flyre\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\flyre\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1597, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
UnicodeDecodeError: 'cp950' codec can't decode byte 0xaf in position 2309: illegal multibyte sequence
Exception in thread Thread-45 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\flyre\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  F

In [ ]:
## patch函釋，偵錯全黑問題

In [10]:
import subprocess
import shutil
from pathlib import Path

def postprocess_final_video(base_plain: Path, settings: dict, out_path: Path, base_w=1920, base_h=1080, base_fps=25, debug=True):
    post = (settings.get("post") or {})
    nothing = True
    if (post.get("color") or {}).get("enabled", False): nothing = False
    if (post.get("denoise") or {}).get("enabled", False): nothing = False
    if (post.get("sharpen") or {}).get("enabled", False): nothing = False
    if (post.get("fps_interp") or {}).get("enabled", False): nothing = False
    for ov in (post.get("overlays") or []):
        if ov.get("enabled", False): nothing = False

    if nothing:
        shutil.copyfile(base_plain, out_path)
        return True, "copy"

    # ----------------------------------------
    # Build filter chain and get extra_inputs (引入你原來的建構函式)
    # ----------------------------------------
    extra_inputs, fc, vout = build_filter_chain_and_inputs(base_w, base_h, settings, base_fps=base_fps)

    if debug:
        print("[Debug] base_plain:", base_plain)
        print("[Debug] out_path:", out_path)
        print("[Debug] extra_inputs:", extra_inputs)
        print("[Debug] filter_complex:", fc)
        print("[Debug] vout:", vout)
    # --- 找尋 webm overlay ---
    webm_overlay = None
    for i in range(len(extra_inputs) - 1):
        if (
            isinstance(extra_inputs[i], str)
            and extra_inputs[i] == "-i"
            and isinstance(extra_inputs[i + 1], str)
            and extra_inputs[i + 1].lower().endswith(".webm")
        ):
            webm_overlay = Path(extra_inputs[i + 1])
            break
    if not webm_overlay:
        for item in extra_inputs:
            if isinstance(item, str) and item.lower().endswith(".webm"):
                webm_overlay = Path(item)
                break

    # ----------------------------------------
    # 【偵錯版本】若發現 filter_complex 有誤，當場 print 並停下
    # ----------------------------------------
    if fc is None or not isinstance(fc, str):
        print("[Error] filter_complex 構建失敗，內容:", fc)
        raise ValueError("filter_complex 構建失敗")
    if "[0:v]" not in fc:
        print("[Error] filter_complex 缺少 base input，[0:v] 必須是主序列！")
        raise ValueError("filter_complex 疊圖序列缺失（main base）。")

    if webm_overlay and "overlay" in fc:
        # 這是進階 overlay 的 PNG 疊圖路徑
        png_pattern, png_dir = convert_webm_overlay_to_pngs(webm_overlay, "adv_overlay", frame_rate=base_fps)
        if debug:
            print(f"[Debug] 進階 overlay模式：webm_overlay = {webm_overlay}")
            print(f"[Debug] png_pattern = {png_pattern}, png_dir = {png_dir}")
        # PNG 疊圖指令
        cmd = [
            "ffmpeg", "-y", "-i", str(base_plain),
            "-framerate", str(base_fps), "-i", png_pattern,
            "-filter_complex", "[0:v][1:v]overlay=0:0:shortest=1[v1]",
            "-map", "[v1]", "-map", "0:a?",
            "-c:v", "libx264", "-pix_fmt", "yuv420p",
            "-c:a", "aac", "-movflags", "+faststart", str(out_path)
        ]
        if debug:
            print("[Debug] ffmpeg overlay command:\n ", " ".join(map(str, cmd)))
        res = subprocess.run(cmd, capture_output=True, text=True)
        print("----ffmpeg stderr (tail)----")
        print((res.stderr or "")[-600:])
        print("----------------------------")
        cleanup_overlay_pngs(png_dir, "adv_overlay")
        ok = (res.returncode == 0 and out_path.exists() and out_path.stat().st_size > 0)
        if not ok:
            print("[Error] ffmpeg 疊圖失敗！returncode=", res.returncode)
            print("stderr last 800:\n", (res.stderr or "")[-800:])
        return ok, (res.stderr or "")[-400:] if not ok else "ok"

    else:
        # 沒有 webm overlay，沿用原底圖與疊圖序列
        cmd = ["ffmpeg", "-y", "-i", str(base_plain)] + extra_inputs + [
            "-filter_complex", fc, "-map", f"[{vout}]", "-map", "0:a?",
            "-c:v", "libx264", "-pix_fmt", "yuv420p", "-c:a", "aac", "-movflags", "+faststart", str(out_path)
        ]
        if debug:
            print("[Debug] ffmpeg non-overlay command:\n ", " ".join(map(str, cmd)))
        res = subprocess.run(cmd, capture_output=True, text=True)
        print("----ffmpeg stderr (tail)----")
        print((res.stderr or "")[-600:])
        print("----------------------------")
        ok = (res.returncode == 0 and out_path.exists() and out_path.stat().st_size > 0)
        if not ok:
            print("[Error] ffmpeg 疊圖失敗！returncode=", res.returncode)
            print("stderr last 800:\n", (res.stderr or "")[-800:])
        return ok, (res.stderr or "")[-400:] if not ok else "ok"

## 底下進行測試為何加特效全黑

In [15]:
import subprocess
import shutil
from pathlib import Path

def postprocess_final_video(base_plain: Path, settings: dict, out_path: Path, base_w=1920, base_h=1080, base_fps=25, debug=True):
    post = (settings.get("post") or {})
    nothing = True
    if (post.get("color") or {}).get("enabled", False): nothing = False
    if (post.get("denoise") or {}).get("enabled", False): nothing = False
    if (post.get("sharpen") or {}).get("enabled", False): nothing = False
    if (post.get("fps_interp") or {}).get("enabled", False): nothing = False
    for ov in (post.get("overlays") or []):
        if ov.get("enabled", False): nothing = False

    if nothing:
        shutil.copyfile(base_plain, out_path)
        return True, "copy"

    # ----------------------------------------
    # Build filter chain and get extra_inputs (引入你原來的建構函式)
    # ----------------------------------------
    extra_inputs, fc, vout = build_filter_chain_and_inputs(base_w, base_h, settings, base_fps=base_fps)

    if debug:
        print("[Debug] base_plain:", base_plain)
        print("[Debug] out_path:", out_path)
        print("[Debug] extra_inputs:", extra_inputs)
        print("[Debug] filter_complex:", fc)
        print("[Debug] vout:", vout)
    # --- 找尋 webm overlay ---
    webm_overlay = None
    for i in range(len(extra_inputs) - 1):
        if (
            isinstance(extra_inputs[i], str)
            and extra_inputs[i] == "-i"
            and isinstance(extra_inputs[i + 1], str)
            and extra_inputs[i + 1].lower().endswith(".webm")
        ):
            webm_overlay = Path(extra_inputs[i + 1])
            break
    if not webm_overlay:
        for item in extra_inputs:
            if isinstance(item, str) and item.lower().endswith(".webm"):
                webm_overlay = Path(item)
                break

    # ----------------------------------------
    # 【偵錯版本】若發現 filter_complex 有誤，當場 print 並停下
    # ----------------------------------------
    if fc is None or not isinstance(fc, str):
        print("[Error] filter_complex 構建失敗，內容:", fc)
        raise ValueError("filter_complex 構建失敗")
    if "[0:v]" not in fc:
        print("[Error] filter_complex 缺少 base input，[0:v] 必須是主序列！")
        raise ValueError("filter_complex 疊圖序列缺失（main base）。")

    if webm_overlay and "overlay" in fc:
        # 這是進階 overlay 的 PNG 疊圖路徑
        png_pattern, png_dir = convert_webm_overlay_to_pngs(webm_overlay, "adv_overlay", frame_rate=base_fps)
        if debug:
            print(f"[Debug] 進階 overlay模式：webm_overlay = {webm_overlay}")
            print(f"[Debug] png_pattern = {png_pattern}, png_dir = {png_dir}")
        # PNG 疊圖指令
        cmd = [
            "ffmpeg", "-y", "-i", str(base_plain),
            "-framerate", str(base_fps), "-i", png_pattern,
            "-filter_complex", "[0:v][1:v]overlay=0:0:shortest=1[v1]",
            "-map", "[v1]", "-map", "0:a?",
            "-c:v", "libx264", "-pix_fmt", "yuv420p",
            "-c:a", "aac", "-movflags", "+faststart", str(out_path)
        ]
        if debug:
            print("[Debug] ffmpeg overlay command:\n ", " ".join(map(str, cmd)))
        res = subprocess.run(cmd, capture_output=True, text=True)
        print("----ffmpeg stderr (tail)----")
        print((res.stderr or "")[-600:])
        print("----------------------------")
        cleanup_overlay_pngs(png_dir, "adv_overlay")
        ok = (res.returncode == 0 and out_path.exists() and out_path.stat().st_size > 0)
        if not ok:
            print("[Error] ffmpeg 疊圖失敗！returncode=", res.returncode)
            print("stderr last 800:\n", (res.stderr or "")[-800:])
        return ok, (res.stderr or "")[-400:] if not ok else "ok"

    else:
        # 沒有 webm overlay，沿用原底圖與疊圖序列
        cmd = ["ffmpeg", "-y", "-i", str(base_plain)] + extra_inputs + [
            "-filter_complex", fc, "-map", f"[{vout}]", "-map", "0:a?",
            "-c:v", "libx264", "-pix_fmt", "yuv420p", "-c:a", "aac", "-movflags", "+faststart", str(out_path)
        ]
        if debug:
            print("[Debug] ffmpeg non-overlay command:\n ", " ".join(map(str, cmd)))
        res = subprocess.run(cmd, capture_output=True, text=True)
        print("----ffmpeg stderr (tail)----")
        print((res.stderr or "")[-600:])
        print("----------------------------")
        ok = (res.returncode == 0 and out_path.exists() and out_path.stat().st_size > 0)
        if not ok:
            print("[Error] ffmpeg 疊圖失敗！returncode=", res.returncode)
            print("stderr last 800:\n", (res.stderr or "")[-800:])
        return ok, (res.stderr or "")[-400:] if not ok else "ok"

## 底下為AI逐步測試

In [24]:
!ffmpeg -y -i base_plain.mp4 -c copy base_test1.mp4

ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --en

In [25]:
!ffmpeg -y -i base_plain.mp4 -i adv_overlay_000001.png -filter_complex "[0:v][1:v]overlay=0:0" -frames:v 1 test_overlay_static.png

ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --en

In [29]:
import os
print(os.getcwd())

C:\Users\flyre\mynotebooks


In [30]:
import os
for f in os.listdir():
    if f.endswith(".png"):
        print(f)

ai_scene_2.png
ai_scene_3.png
ai_scene_3_gen_0953b.png
ai_scene_4.png
dalle3_chinese_scene.png
dalle3_chinese_scene_1920x1080.png
dalle3_chinese_scene_20251031_042326.png
dalle3_chinese_scene_20251031_052141.png
dalle3_chinese_scene_20251031_052141_1920x1080.png
dalle3_chinese_scene_20251101_040029.png
dalle3_chinese_scene_20251101_040029_1920x1080.png
dalle3_chinese_scene_20251101_042731.png
dalle3_chinese_scene_20251101_042731_1920x1080.png
dalle3_chinese_scene_20251101_043524.png
dalle3_chinese_scene_20251101_043524_1920x1080.png
dalle3_chinese_scene_20251101_044234.png
dalle3_chinese_scene_20251101_044234_1920x1080.png
dalle3_一個人_流星_20251031_042326_1920x1080.png
dalle_background.png
pillow_test.png
rainline_gray_64.png
scene_001_img.png
Screenshot 2025-10-11 221037.png
snowflake_lightblue_64.png
test_caption.png
_adv_tex_leaves.png
_adv_tex_snow.png
樹林瀑布.png


In [31]:
import glob
print(glob.glob("*.webm"))

['_adv_overlay_L1_1592920c30.webm', '_adv_overlay_L1_dd4dda8988.webm', '_adv_overlay_L2_9abd9334e0.webm']


In [32]:
!ffmpeg -y -i _adv_overlay_L1_1592920c30.webm -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" _overlaytemp/adv_overlay_%06d.png

ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --en

In [35]:
!ffmpeg -y -i base_plain.mp4 -i _overlaytemp/adv_overlay_000001.png -filter_complex "[0:v][1:v]overlay=0:0" -frames:v 1 test_overlay_static.png

ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --en

In [36]:
from PIL import Image

img = Image.open("_overlaytemp/adv_overlay_000001.png")  # 依你的png實際路徑
print("mode:", img.mode)
img.show()
if "A" in img.mode:
    img_alpha = img.getchannel("A")
    img_alpha.show()

mode: RGB


In [37]:
!ffmpeg -hide_banner -i _adv_overlay_L1_1592920c30.webm

Input #0, matroska,webm, from '_adv_overlay_L1_1592920c30.webm':
  Metadata:
    ENCODER         : Lavf62.3.100
  Duration: 00:00:12.00, start: 0.000000, bitrate: 12935 kb/s
  Stream #0:0: Video: vp9 (Profile 0), yuv420p(tv, progressive), 1920x1080, SAR 1:1 DAR 16:9, 25 fps, 25 tbr, 1k tbn
    Metadata:
      alpha_mode      : 1
      ENCODER         : Lavc62.11.100 libvpx-vp9
      DURATION        : 00:00:12.000000000
At least one output file must be specified


In [42]:
from PIL import Image
img = Image.open('花瓣透明圖.png')
print(img.mode)  # RGBA 代表有透明通道，RGB 沒有

RGBA


In [1]:
import os

main_dir = r"C:/Users/flyre/mynotebooks"
os.chdir(main_dir)
os.makedirs("frames", exist_ok=True)

# 拆幀，寬640，高度自動等比例
!ffmpeg -y -i base_plain.mp4 -vf "scale=640:-1" frames/base_%04d.png

ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --en

In [4]:
from PIL import Image

petal = Image.open('花瓣透明圖.png')
print('petal.png mode:', petal.mode)
petal.show()

petal.png mode: RGBA


In [5]:
import numpy as np

petal = Image.open('花瓣透明圖.png').convert('RGBA')
arr = np.array(petal)
print('左上alpha:', arr[0,0,3])  # [0,0,3] 是左上pixel的alpha
print('中間alpha:', arr[arr.shape[0]//2, arr.shape[1]//2, 3])  # 圖中央

左上alpha: 0
中間alpha: 255


In [6]:
arr = np.array(petal)
# 只要alpha==0就該是背景，設成假紅色檢查
arr[(arr[:,:,3]==0)] = [255,0,0,255]
Image.fromarray(arr).show()

In [7]:
import glob
from PIL import Image
import os

os.makedirs("out1", exist_ok=True)
frame_files = sorted(glob.glob('frames/base_*.png'))

for fn in frame_files:
    frame = Image.open(fn).convert('RGBA')
    frame_out = frame.copy()
    petal = Image.open('花瓣透明圖.png').convert('RGBA')
    frame_out.paste(petal, (60, 120), petal)
    outname = 'out1/' + os.path.basename(fn)
    frame_out.save(outname)

In [8]:
import glob
from PIL import Image
import os

os.makedirs("out2", exist_ok=True)
frame_files = sorted(glob.glob('out1/base_*.png'))

for fn in frame_files:
    img = Image.open(fn).convert('RGBA')
    petal2 = Image.open('花瓣透明圖.png').convert('RGBA')
    petal2.putalpha(int(255*0.2))  # 透明度設 0.2
    img.paste(petal2, (230, 250), petal2)  # 疊上淡花瓣（不同位置）
    outname = 'out2/' + os.path.basename(fn)
    img.save(outname)

In [9]:
# 產生 MP4（不帶透明度，只供一般觀看）
!ffmpeg -y -framerate 30 -i out2/base_%04d.png -c:v libx264 -pix_fmt yuv420p demo_out.mp4

# 產生 WebM（帶透明度的格式，僅 VP8/VP9 + yuva420p 可做到）
!ffmpeg -y -framerate 30 -i out2/base_%04d.png -c:v libvpx-vp9 -pix_fmt yuva420p demo_out.webm

ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --en